In [1]:
import hail as hl

Loading BokehJS ...

In [2]:
hl.init(worker_memory="highmem", driver_memory='highmem') 


/opt/anaconda3/envs/hail/lib/python3.9/site-packages/hail/backend/service_backend.py:563: UserWarning:

Modifying the requester pays project or buckets at runtime using flags is deprecated. Expect this behavior to become unsupported soon.

Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.133-4c60fddb171a
LOGGING: writing to /Users/tk508/HAIL/Genetics-Gym/my_gg/fixing_VSMs/hail-20251004-1440-0.2.133-4c60fddb171a.log


In [6]:

vsm_ens = {
    'esm1b': {'path': 'gs://genetics-gym-not-public/Trisha/updated_data_tables/esm1b_scores.ht', 
              'score_cols': ['esm1b']
             },
    'misfit': {'path': 'gs://genetics-gym-not-public/Trisha/updated_data_tables/misfit_scores.ht', 
               'score_cols': ['MisFit_D', 'MisFit_S']
              },
    'AM': {'path': 'gs://genetics-gym-not-public/Trisha/hail_VSM_tables_updated/AM_fixed.ht', 
           'score_cols': ['AM']
           }, 
    'RASP': {'path': 'gs://missense-scoring/mutation/rasp_scores.ht', 
             'score_cols': ['rasp_score']
            },
    'MPC': {'path': 'gs://missense-scoring/mpc_grch38_deduped_with_outliers_2024-04-30.ht', 
             'score_cols': ['mpc']
            },
    'ProteinMPNN': {'path': 'gs://genetics-gym-not-public/Trisha/hail_VSM_tables_updated/protein_mpnn.ht', 
             'score_cols': ['proteinmpnn_llr']
            }, 
    'AM_canon': {'path': 'gs://genetics-gym-not-public/Trisha/hail_VSM_tables_updated/AM_all_aa_sub_uniprot_canon.ht',
                 'score_cols': ['AM']

    },
    'AM_noncanon': {'path': 'gs://genetics-gym-not-public/Trisha/hail_VSM_tables_updated/AM_aa_noncanon_enst.ht', 
                    'score_cols': ['AM']

    },
    'PAI3D': {'path': 'gs://genetics-gym-not-public/Trisha/hail_VSM_tables_updated/PAI3D.ht', 
          'score_cols': ['score_PAI3D']
             }
    
}

# Check completeness method

In [39]:
def check_score_completenes(ht, score_name): 
    check_ht = ht
    # check_ht = ht.key_by('locus', 'alleles')
    # check_ht = check_ht.collect_by_key()
    # check_ht = check_ht.checkpoint(f'gs://trisha-tmp/new_VSM_temp/{score_name}_collected.ht', overwrite=True)
    check_ht = check_ht.annotate(
        score_mane = hl.find(lambda x: x.mane_select == True, check_ht.values)[score_name],
        is_mane = hl.find(lambda x: x.mane_select == True, check_ht.values)['mane_select'],
        score_canon =  hl.find(lambda x: x.canonical == True, check_ht.values)[score_name],
        is_canonical = hl.find(lambda x: x.canonical == True, check_ht.values)['canonical'],
        score_any = hl.find(lambda x: hl.is_defined(x[score_name]) == True, check_ht.values)[score_name]

    )  
    check_htf1 = check_ht.filter(hl.is_defined(check_ht.score_mane))
    check_htf2 = check_ht.filter(hl.is_defined(check_ht.score_canon))
    check_htf3 = check_ht.filter(hl.is_defined(check_ht.score_any))
    mane_count = check_htf1.count()
    canon_count = check_htf2.count()
    any_count = check_htf3.count()
    count_results = {'mane': mane_count, 'canon': canon_count, 'any': any_count}
    return count_results


In [ ]:
vsm_ht = hl.read_table('gs://trisha-tmp/make_linker/linker2_temp_union_final.ht')
vsm_ht = vsm_ht.annotate(
    uniprot_base = hl.if_else(
        vsm_ht.uniprot_em_id.contains('-'), vsm_ht.uniprot_em_id.split('-')[0], hl.missing(hl.tstr)
    )
)

Output()

Output()

In [22]:
vsm_ht.show()

NameError: name 'vsm_ht' is not defined

# ESM1b

In [5]:
esm1b_ht = hl.read_table('gs://genetics-gym-not-public/Trisha/updated_data_tables/esm1b_scores.ht')


In [16]:
esm1b_ht.show()

,,,,
aa_alt,aa_pos,aa_ref,esm_score,uniprot_isoform
str,int32,str,float64,str
"""A""",1,"""M""",-6.09e+00,"""A0A024RBG1"""
"""C""",1,"""M""",-7.72e+00,"""A0A024RBG1"""
"""D""",1,"""M""",-6.25e+00,"""A0A024RBG1"""
"""E""",1,"""M""",-5.44e+00,"""A0A024RBG1"""
"""F""",1,"""M""",-6.36e+00,"""A0A024RBG1"""
"""G""",1,"""M""",-6.06e+00,"""A0A024RBG1"""
"""H""",1,"""M""",-7.20e+00,"""A0A024RBG1"""
"""I""",1,"""M""",-6.13e+00,"""A0A024RBG1"""


In [7]:
ht = hl.read_table(vsm_ens['esm1b']['path'])
ht = ht.key_by('uniprot_isoform', 'aa_pos', 'aa_ref', 'aa_alt')
ht = ht.checkpoint('gs://genetics-gym-not-public/Trisha/hail_VSM_tables_updated/esm1b_keyed.ht')
vsm_ht = vsm_ht.annotate(
    esm1b_base = ht[vsm_ht.uniprot_base, vsm_ht.aa_pos, vsm_ht.aa_ref, vsm_ht.aa_alt].esm_score,
    esm1b_iso = ht[vsm_ht.uniprot_em_id, vsm_ht.aa_pos, vsm_ht.aa_ref, vsm_ht.aa_alt].esm_score
)
vsm_ht = vsm_ht.checkpoint('gs://trisha-tmp/new_VSM_temp/esm1b_joined.ht')


2025-09-30 11:04:27.735 Hail: INFO: Coerced sorted dataset==(42286 + 2) / 42286]
2025-09-30 11:04:28.658 Hail: INFO: Coerced dataset with out-of-order partitions.
2025-09-30 11:30:22.926 Hail: INFO: wrote table with 486678080 rows in 42286 partitions to gs://genetics-gym-not-public/Trisha/hail_VSM_tables_updated/esm1b_keyed.ht
2025-09-30 11:30:58.171 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-09-30 11:50:55.678 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-09-30 12:27:24.932 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-09-30 12:45:44.643 Hail: INFO: wrote table with 273113217 rows in 203 partitions to gs://trisha-tmp/new_VSM_temp/esm1b_joined.ht


In [15]:
vsm_ht.show()

,,,,,,,,,,,,,,
enst,locus,alleles,aa_pos,aa_ref,aa_alt,gene_symbol,ensp,ensg,mane_NM,mane_select,canonical,uniprot_em_id,transcript_mane_select,uniprot_base
str,locus<GRCh38>,array<str>,int32,str,str,str,str,str,str,bool,bool,str,bool,str
"""ENST00000000233""",chr7:127588502,"[""G"",""A""]",2,"""G""","""S""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA
"""ENST00000000233""",chr7:127588502,"[""G"",""C""]",2,"""G""","""R""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA
"""ENST00000000233""",chr7:127588502,"[""G"",""T""]",2,"""G""","""C""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA
"""ENST00000000233""",chr7:127588503,"[""G"",""A""]",2,"""G""","""D""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA
"""ENST00000000233""",chr7:127588503,"[""G"",""C""]",2,"""G""","""A""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA
"""ENST00000000233""",chr7:127588503,"[""G"",""T""]",2,"""G""","""V""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA
"""ENST00000000233""",chr7:127588505,"[""C"",""A""]",3,"""L""","""I""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA
"""ENST00000000233""",chr7:127588505,"[""C"",""G""]",3,"""L""","""V""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA


In [14]:
## V2
vsm_ht = vsm_ht.repartition(500)

In [16]:

vsm_ht = vsm_ht.key_by('uniprot_em_id', 'aa_pos', 'aa_ref', 'aa_alt')
ht = hl.read_table('gs://genetics-gym-not-public/Trisha/hail_VSM_tables_updated/esm1b_keyed.ht')

vsm_ht_j1 = vsm_ht.join(ht, how='left')
vsm_ht_j1 = vsm_ht_j1.checkpoint('gs://trisha-tmp/new_VSM_temp/esm1b_lj1.ht')

2025-09-30 16:06:11.936 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-09-30 16:53:19.022 Hail: INFO: wrote table with 273113217 rows in 500 partitions to gs://trisha-tmp/new_VSM_temp/esm1b_lj1.ht


In [19]:
vsm_ht_j2 = vsm_ht_j1.key_by('uniprot_base', 'aa_pos', 'aa_ref', 'aa_alt')
vsm_ht_j2 = vsm_ht_j2.join(ht, how='left')
vsm_ht_j2 = vsm_ht_j2.checkpoint('gs://trisha-tmp/new_VSM_temp/esm1b_lj2.ht', overwrite=True)

2025-09-30 18:13:53.951 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'esm_score' -> 'esm_score_1'
2025-09-30 18:16:28.969 Hail: INFO: Ordering unsorted dataset with network shuffle
2025-09-30 18:42:17.680 Hail: INFO: wrote table with 273113217 rows in 500 partitions to gs://trisha-tmp/new_VSM_temp/esm1b_lj2.ht


In [4]:
check_ht = hl.read_table('gs://trisha-tmp/new_VSM_temp/esm1b_lj2.ht')

Output()

Output()

In [21]:
check_ht.show()

,,,,,,,,,,,,,,,,
uniprot_base,aa_pos,aa_ref,aa_alt,uniprot_em_id,enst,locus,alleles,gene_symbol,ensp,ensg,mane_NM,mane_select,canonical,transcript_mane_select,esm_score,esm_score_1
str,int32,str,str,str,str,locus<GRCh38>,array<str>,str,str,str,str,bool,bool,bool,float64,float64
"""A0A096LP49""",2,"""P""","""A""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303827,"[""G"",""C""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-2.48e+00
"""A0A096LP49""",2,"""P""","""L""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303826,"[""G"",""A""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-4.21e+00
"""A0A096LP49""",2,"""P""","""Q""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303826,"[""G"",""T""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-3.84e+00
"""A0A096LP49""",2,"""P""","""R""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303826,"[""G"",""C""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-3.64e+00
"""A0A096LP49""",2,"""P""","""S""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303827,"[""G"",""A""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-3.29e+00
"""A0A096LP49""",2,"""P""","""T""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303827,"[""G"",""T""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-4.07e+00
"""A0A096LP49""",2,"""W""","""C""","""A0A096LP49-2""","""ENST00000569961""",chr9:136303137,"[""C"",""A""]","""CCDC187""","""ENSP00000456543""","""ENSG00000260220""",NA,False,False,False,-4.32e+00,NA
"""A0A096LP49""",2,"""W""","""C""","""A0A096LP49-2""","""ENST00000569961""",chr9:136303137,"[""C"",""G""]","""CCDC187""","""ENSP00000456543""","""ENSG00000260220""",NA,False,False,False,-4.32e+00,NA


In [5]:
check_ht = check_ht.annotate(
    esm_comb = hl.coalesce(check_ht.esm_score, check_ht.esm_score_1)
)

In [23]:
check_ht.show()

,,,,,,,,,,,,,,,,,
uniprot_base,aa_pos,aa_ref,aa_alt,uniprot_em_id,enst,locus,alleles,gene_symbol,ensp,ensg,mane_NM,mane_select,canonical,transcript_mane_select,esm_score,esm_score_1,esm_comb
str,int32,str,str,str,str,locus<GRCh38>,array<str>,str,str,str,str,bool,bool,bool,float64,float64,float64
"""A0A096LP49""",2,"""P""","""A""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303827,"[""G"",""C""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-2.48e+00,-2.48e+00
"""A0A096LP49""",2,"""P""","""L""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303826,"[""G"",""A""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-4.21e+00,-4.21e+00
"""A0A096LP49""",2,"""P""","""Q""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303826,"[""G"",""T""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-3.84e+00,-3.84e+00
"""A0A096LP49""",2,"""P""","""R""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303826,"[""G"",""C""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-3.64e+00,-3.64e+00
"""A0A096LP49""",2,"""P""","""S""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303827,"[""G"",""A""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-3.29e+00,-3.29e+00
"""A0A096LP49""",2,"""P""","""T""","""A0A096LP49-1""","""ENST00000624277""",chr9:136303827,"[""G"",""T""]","""CCDC187""","""ENSP00000485393""","""ENSG00000260220""",NA,False,False,False,NA,-4.07e+00,-4.07e+00
"""A0A096LP49""",2,"""W""","""C""","""A0A096LP49-2""","""ENST00000569961""",chr9:136303137,"[""C"",""A""]","""CCDC187""","""ENSP00000456543""","""ENSG00000260220""",NA,False,False,False,-4.32e+00,NA,-4.32e+00
"""A0A096LP49""",2,"""W""","""C""","""A0A096LP49-2""","""ENST00000569961""",chr9:136303137,"[""C"",""G""]","""CCDC187""","""ENSP00000456543""","""ENSG00000260220""",NA,False,False,False,-4.32e+00,NA,-4.32e+00


In [24]:
check_ht_f = check_ht.filter((~hl.is_defined(check_ht.esm_comb)) & (check_ht.mane_select))

In [25]:
check_ht_f.show()

,,,,,,,,,,,,,,,,,
uniprot_base,aa_pos,aa_ref,aa_alt,uniprot_em_id,enst,locus,alleles,gene_symbol,ensp,ensg,mane_NM,mane_select,canonical,transcript_mane_select,esm_score,esm_score_1,esm_comb
str,int32,str,str,str,str,locus<GRCh38>,array<str>,str,str,str,str,bool,bool,bool,float64,float64,float64
"""A0PJE2""",2,"""N""","""D""","""A0PJE2-4""","""ENST00000444610""",chr13:51799656,"[""T"",""C""]","""DHRS12""","""ENSP00000411565""","""ENSG00000102796""","""NM_001377533.1""",True,True,True,NA,NA,NA
"""A0PJE2""",2,"""N""","""H""","""A0PJE2-4""","""ENST00000444610""",chr13:51799656,"[""T"",""G""]","""DHRS12""","""ENSP00000411565""","""ENSG00000102796""","""NM_001377533.1""",True,True,True,NA,NA,NA
"""A0PJE2""",2,"""N""","""I""","""A0PJE2-4""","""ENST00000444610""",chr13:51799655,"[""T"",""A""]","""DHRS12""","""ENSP00000411565""","""ENSG00000102796""","""NM_001377533.1""",True,True,True,NA,NA,NA
"""A0PJE2""",2,"""N""","""K""","""A0PJE2-4""","""ENST00000444610""",chr13:51799654,"[""A"",""C""]","""DHRS12""","""ENSP00000411565""","""ENSG00000102796""","""NM_001377533.1""",True,True,True,NA,NA,NA
"""A0PJE2""",2,"""N""","""K""","""A0PJE2-4""","""ENST00000444610""",chr13:51799654,"[""A"",""T""]","""DHRS12""","""ENSP00000411565""","""ENSG00000102796""","""NM_001377533.1""",True,True,True,NA,NA,NA
"""A0PJE2""",2,"""N""","""S""","""A0PJE2-4""","""ENST00000444610""",chr13:51799655,"[""T"",""C""]","""DHRS12""","""ENSP00000411565""","""ENSG00000102796""","""NM_001377533.1""",True,True,True,NA,NA,NA
"""A0PJE2""",2,"""N""","""T""","""A0PJE2-4""","""ENST00000444610""",chr13:51799655,"[""T"",""G""]","""DHRS12""","""ENSP00000411565""","""ENSG00000102796""","""NM_001377533.1""",True,True,True,NA,NA,NA
"""A0PJE2""",2,"""N""","""Y""","""A0PJE2-4""","""ENST00000444610""",chr13:51799656,"[""T"",""A""]","""DHRS12""","""ENSP00000411565""","""ENSG00000102796""","""NM_001377533.1""",True,True,True,NA,NA,NA


In [26]:
uni_missing = check_ht_f.aggregate(hl.agg.collect_as_set(check_ht_f.uniprot_base))

In [27]:
len(uni_missing)

163

In [ ]:
# need to get which uniprot are fully defined with other isoforms....

In [28]:
check_ht.describe()

----------------------------------------
Global fields:
    'grange': array<int32> 
    'vep_help': str 
    'vep_config': str 
    'version': str 
----------------------------------------
Row fields:
    'uniprot_base': str 
    'aa_pos': int32 
    'aa_ref': str 
    'aa_alt': str 
    'uniprot_em_id': str 
    'enst': str 
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'gene_symbol': str 
    'ensp': str 
    'ensg': str 
    'mane_NM': str 
    'mane_select': bool 
    'canonical': bool 
    'transcript_mane_select': bool 
    'esm_score': float64 
    'esm_score_1': float64 
    'esm_comb': float64 
----------------------------------------
Key: ['uniprot_base', 'aa_pos', 'aa_ref', 'aa_alt']
----------------------------------------


In [6]:
check_ht_lk = check_ht.key_by('locus', 'alleles')

In [30]:
check_ht_lk.show()

,,,,,,,,,,,,,,,,,
uniprot_base,aa_pos,aa_ref,aa_alt,uniprot_em_id,enst,locus,alleles,gene_symbol,ensp,ensg,mane_NM,mane_select,canonical,transcript_mane_select,esm_score,esm_score_1,esm_comb
str,int32,str,str,str,str,locus<GRCh38>,array<str>,str,str,str,str,bool,bool,bool,float64,float64,float64
NA,2,"""K""","""Q""","""A0A2U3U0J3""","""ENST00000641515""",chr1:65568,"[""A"",""C""]","""OR4F5""","""ENSP00000493376""","""ENSG00000186092""","""NM_001005484.2""",True,True,True,NA,NA,NA
NA,2,"""K""","""E""","""A0A2U3U0J3""","""ENST00000641515""",chr1:65568,"[""A"",""G""]","""OR4F5""","""ENSP00000493376""","""ENSG00000186092""","""NM_001005484.2""",True,True,True,NA,NA,NA
NA,2,"""K""","""T""","""A0A2U3U0J3""","""ENST00000641515""",chr1:65569,"[""A"",""C""]","""OR4F5""","""ENSP00000493376""","""ENSG00000186092""","""NM_001005484.2""",True,True,True,NA,NA,NA
NA,2,"""K""","""R""","""A0A2U3U0J3""","""ENST00000641515""",chr1:65569,"[""A"",""G""]","""OR4F5""","""ENSP00000493376""","""ENSG00000186092""","""NM_001005484.2""",True,True,True,NA,NA,NA
NA,2,"""K""","""M""","""A0A2U3U0J3""","""ENST00000641515""",chr1:65569,"[""A"",""T""]","""OR4F5""","""ENSP00000493376""","""ENSG00000186092""","""NM_001005484.2""",True,True,True,NA,NA,NA
NA,2,"""K""","""N""","""A0A2U3U0J3""","""ENST00000641515""",chr1:65570,"[""G"",""C""]","""OR4F5""","""ENSP00000493376""","""ENSG00000186092""","""NM_001005484.2""",True,True,True,NA,NA,NA
NA,2,"""K""","""N""","""A0A2U3U0J3""","""ENST00000641515""",chr1:65570,"[""G"",""T""]","""OR4F5""","""ENSP00000493376""","""ENSG00000186092""","""NM_001005484.2""",True,True,True,NA,NA,NA
NA,3,"""K""","""Q""","""A0A2U3U0J3""","""ENST00000641515""",chr1:65571,"[""A"",""C""]","""OR4F5""","""ENSP00000493376""","""ENSG00000186092""","""NM_001005484.2""",True,True,True,NA,NA,NA


In [7]:
check_ht_lk_collect = check_ht_lk.collect_by_key()

In [32]:
check_ht_lk_collect.show()

2025-09-30 19:12:15.321 Hail: INFO: Ordering unsorted dataset with network shuffle


,,
locus,alleles,values
locus<GRCh38>,array<str>,"array<struct{uniprot_base: str, aa_pos: int32, aa_ref: str, aa_alt: str, uniprot_em_id: str, enst: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, transcript_mane_select: bool, esm_score: float64, esm_score_1: float64, esm_comb: float64}>"
chr1:65568,"[""A"",""C""]","[(NA,2,""K"",""Q"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]"
chr1:65568,"[""A"",""G""]","[(NA,2,""K"",""E"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]"
chr1:65569,"[""A"",""C""]","[(NA,2,""K"",""T"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]"
chr1:65569,"[""A"",""G""]","[(NA,2,""K"",""R"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]"
chr1:65569,"[""A"",""T""]","[(NA,2,""K"",""M"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]"
chr1:65570,"[""G"",""C""]","[(NA,2,""K"",""N"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]"
chr1:65570,"[""G"",""T""]","[(NA,2,""K"",""N"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]"
chr1:65571,"[""A"",""C""]","[(NA,3,""K"",""Q"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]"


In [8]:
check_ht_lk_collect = check_ht_lk_collect.annotate(
    mane_select = check_ht_lk_collect.values.mane_select, 
    esm_c = check_ht_lk_collect.values.esm_comb

)


In [10]:
check_ht_lk_collect.show()

Output()

,,,,
locus,alleles,values,mane_select,esm_c
locus<GRCh38>,array<str>,"array<struct{uniprot_base: str, aa_pos: int32, aa_ref: str, aa_alt: str, uniprot_em_id: str, enst: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, transcript_mane_select: bool, esm_score: float64, esm_score_1: float64, esm_comb: float64}>",array<bool>,array<float64>
chr1:65568,"[""A"",""C""]","[(NA,2,""K"",""Q"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65568,"[""A"",""G""]","[(NA,2,""K"",""E"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65569,"[""A"",""C""]","[(NA,2,""K"",""T"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65569,"[""A"",""G""]","[(NA,2,""K"",""R"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65569,"[""A"",""T""]","[(NA,2,""K"",""M"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65570,"[""G"",""C""]","[(NA,2,""K"",""N"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65570,"[""G"",""T""]","[(NA,2,""K"",""N"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65571,"[""A"",""C""]","[(NA,3,""K"",""Q"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]


In [9]:
check_ht_lk_collect = check_ht_lk_collect.checkpoint('gs://trisha-tmp/new_VSM_temp/esm1b_lj2_collected.ht')

Output()

A transient error occured. We will automatically retry. Do not be alarmed. We have thus far seen 2 transient errors
(next delay: 2.939s). The most recent error was <class 'hailtop.httpx.ClientResponseError'> 503, message='Service 
Unavailable', url=URL('https://batch.hail.is/api/v1alpha/batches/8323056') body='upstream connect error or 
disconnect/reset before headers. reset reason: connection failure'. 
execute(...): ]8;id=968592;https://batch.hail.is/batches/8323056\8323056]8;;\ ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 100% 2410/2411 -:--:-- 0:12:28

In [ ]:
collect = hl.read_table('gs://trisha-tmp/new_VSM_temp/esm1b_lj2_collected.ht')

In [11]:
check_ht_lk_collect = check_ht_lk_collect.annotate(
    esm_mane = hl.find(lambda x: x.mane_select == True, check_ht_lk_collect.values)['esm_comb']
)


In [12]:
check_ht_lk_collect.show()

Output()

,,,,,
locus,alleles,values,mane_select,esm_c,esm_mane
locus<GRCh38>,array<str>,"array<struct{uniprot_base: str, aa_pos: int32, aa_ref: str, aa_alt: str, uniprot_em_id: str, enst: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, transcript_mane_select: bool, esm_score: float64, esm_score_1: float64, esm_comb: float64}>",array<bool>,array<float64>,float64
chr1:65568,"[""A"",""C""]","[(NA,2,""K"",""Q"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA
chr1:65568,"[""A"",""G""]","[(NA,2,""K"",""E"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA
chr1:65569,"[""A"",""C""]","[(NA,2,""K"",""T"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA
chr1:65569,"[""A"",""G""]","[(NA,2,""K"",""R"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA
chr1:65569,"[""A"",""T""]","[(NA,2,""K"",""M"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA
chr1:65570,"[""G"",""C""]","[(NA,2,""K"",""N"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA
chr1:65570,"[""G"",""T""]","[(NA,2,""K"",""N"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA
chr1:65571,"[""A"",""C""]","[(NA,3,""K"",""Q"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA


In [ ]:
check_ht_lk_collect = check_ht_lk_collect.annotate(
    has_mane = hl.find(lambda x: x.mane_select == True, check_ht_lk_collect.values)['mane_select']
)


In [14]:
check_ht_lk_collect.show()

Output()

,,,,,,
locus,alleles,values,mane_select,esm_c,esm_mane,has_mane
locus<GRCh38>,array<str>,"array<struct{uniprot_base: str, aa_pos: int32, aa_ref: str, aa_alt: str, uniprot_em_id: str, enst: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, transcript_mane_select: bool, esm_score: float64, esm_score_1: float64, esm_comb: float64}>",array<bool>,array<float64>,float64,bool
chr1:65568,"[""A"",""C""]","[(NA,2,""K"",""Q"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA,True
chr1:65568,"[""A"",""G""]","[(NA,2,""K"",""E"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA,True
chr1:65569,"[""A"",""C""]","[(NA,2,""K"",""T"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA,True
chr1:65569,"[""A"",""G""]","[(NA,2,""K"",""R"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA,True
chr1:65569,"[""A"",""T""]","[(NA,2,""K"",""M"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA,True
chr1:65570,"[""G"",""C""]","[(NA,2,""K"",""N"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA,True
chr1:65570,"[""G"",""T""]","[(NA,2,""K"",""N"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA,True
chr1:65571,"[""A"",""C""]","[(NA,3,""K"",""Q"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA],NA,True


In [15]:
collect_has = check_ht_lk_collect.filter(hl.is_defined(check_ht_lk_collect.esm_mane))

In [16]:
collect_has.count()

Output()

65780956

In [17]:
collect_doesnt = check_ht_lk_collect.filter(~hl.is_defined(check_ht_lk_collect.esm_mane))

In [18]:
collect_doesnt.count()

Output()

12666939

In [19]:
collect_has_mane = collect_has.filter(collect_has.has_mane)

In [20]:
collect_has_mane.count()

Output()

65780956

# Protein MPNN

In [4]:
# load linker
vsm_ht = hl.read_table('gs://trisha-tmp/make_linker/linker2_temp_union_final.ht')
vsm_ht = vsm_ht.annotate(
    uniprot_base = hl.if_else(
        vsm_ht.uniprot_em_id.contains('-'), vsm_ht.uniprot_em_id.split('-')[0], hl.missing(hl.tstr)
    )
)

Output()

Output()

In [ ]:
ht = hl.read_table(vsm_ens['ProteinMPNN']['path'])
# ht is already keyed by id, pos, ref, alt
# vsm_ht = vsm_ht.key_by('uniprot_base', 'aa_pos', 'aa_ref', 'aa_alt')
# vsm_ht = vsm_ht.join(ht, how='left')
# joined = ht[vsm_ht.uniprot_base, vsm_ht.aa_pos, vsm_ht.aa_ref, vsm_ht.aa_alt]
# vsm_ht = vsm_ht.annotate(
#     proteinmpnn_llr = joined.proteinmpnn_llr
# )

In [25]:
ht = hl.read_table(vsm_ens['ProteinMPNN']['path'])


Output()

Output()

In [26]:
ht.show()

Output()

,,,,,,,
AF_structure_name,uniprot_id,uniprot_aa_pos,uniprot_aa_ref,uniprot_aa_alt,ref_log_p,alt_log_p,proteinmpnn_llr
str,str,int32,str,str,float64,float64,float64
"""AF-A0A024R1R8-F1-model_v4""","""A0A024R1R8""",1,"""M""","""A""",-2.66e+00,-2.39e+00,2.69e-01
"""AF-A0A024R1R8-F1-model_v4""","""A0A024R1R8""",1,"""M""","""C""",-2.66e+00,-4.95e+00,-2.29e+00
"""AF-A0A024R1R8-F1-model_v4""","""A0A024R1R8""",1,"""M""","""D""",-2.66e+00,-3.05e+00,-3.94e-01
"""AF-A0A024R1R8-F1-model_v4""","""A0A024R1R8""",1,"""M""","""E""",-2.66e+00,-2.46e+00,1.96e-01
"""AF-A0A024R1R8-F1-model_v4""","""A0A024R1R8""",1,"""M""","""F""",-2.66e+00,-3.51e+00,-8.46e-01
"""AF-A0A024R1R8-F1-model_v4""","""A0A024R1R8""",1,"""M""","""G""",-2.66e+00,-2.93e+00,-2.72e-01
"""AF-A0A024R1R8-F1-model_v4""","""A0A024R1R8""",1,"""M""","""H""",-2.66e+00,-3.44e+00,-7.82e-01
"""AF-A0A024R1R8-F1-model_v4""","""A0A024R1R8""",1,"""M""","""I""",-2.66e+00,-2.97e+00,-3.15e-01


## Mapping for uniprot isoform

In [7]:
af2db = hl.import_table('gs://genetics-gym-not-public/Emily/af2db_uniprot_final_sm.csv', 
                    delimiter=',', missing = '',
                    types={'uniprot_base': hl.tstr, 
                    'uniprot_isoform': hl.tstr,  
                    'AF2DB': hl.tbool}
                    )
af_ht = af2db.filter(af2db.AF2DB)               
af_ht = af_ht.select('uniprot_isoform', 'uniprot_id')
af_ht = af_ht.key_by('uniprot_id')

Output()

In [ ]:
af_ht.show()



Output()

,
uniprot_isoform,uniprot_id
str,str
"""A0A024R1R8""","""A0A024R1R8"""
"""A0A024RBG1""","""A0A024RBG1"""
"""A0A075B6H7""","""A0A075B6H7"""
"""A0A075B6H8""","""A0A075B6H8"""
"""A0A075B6H9""","""A0A075B6H9"""
"""A0A075B6I0""","""A0A075B6I0"""
"""A0A075B6I1""","""A0A075B6I1"""
"""A0A075B6I3""","""A0A075B6I3"""


In [9]:
af_ht = af_ht.key_by('uniprot_id')

In [10]:
af_ht.write('gs://genetics-gym-not-public/Emily/af2db_uniprot_final.ht')

Output()

## Map the isoform to MPNN

In [43]:

ht = hl.read_table('gs://missense-scoring/mpnn-outputs/AF_total_variants.ht')
af_ht = hl.read_table('gs://genetics-gym-not-public/Emily/af2db_uniprot_final.ht')



Output()

Output()

Output()

Output()

In [44]:
ht = ht.key_by('uniprot')
ht = ht.join(af_ht, how='left')

In [45]:
ht = ht.key_by('uniprot_isoform', 'aa_pos', 'aa_ref', 'aa_alt')
ht = ht.select('proteinmpnn_llr')
ht.write('gs://genetics-gym-not-public/Trisha/hail_VSM_tables_updated/protein_mpnn.ht', overwrite=True)

Output()

## Map MPNN to VSM

In [57]:
vsm_ht = hl.read_table('gs://trisha-tmp/make_linker/linker2_temp_union_final.ht')
vsm_ht = vsm_ht.annotate(
    uniprot_base = hl.if_else(
        vsm_ht.uniprot_em_id.contains('-'), vsm_ht.uniprot_em_id.split('-')[0], hl.missing(hl.tstr)
    )
)
vsm_ht = vsm_ht.key_by('uniprot_em_id', 'aa_pos', 'aa_ref', 'aa_alt')
vsm_ht = vsm_ht.checkpoint(f'gs://trisha-tmp/new_VSM_temp/vsm_uniprot_iso_keyed.ht')

Output()

Output()

Output()

In [58]:
ht = hl.read_table(vsm_ens['ProteinMPNN']['path'])
# ht is already keyed by iso, pos, ref, alt
vsm_ht = vsm_ht.join(ht, how='left')


Output()

Output()

In [59]:
vsm_ht = vsm_ht.checkpoint(f'gs://trisha-tmp/new_VSM_temp/vsm_proteinmpnn.ht', overwrite=True)

Output()

In [62]:
vsm_ht = hl.read_table('gs://trisha-tmp/new_VSM_temp/vsm_proteinmpnn.ht')

Output()

Output()

In [63]:
mane_count, canon_count = check_score_completenes(vsm_ht, 'proteinmpnn_llr')
print('mane: ', mane_count)
print('canon: ', canon_count)

Output()

Output()

Output()

mane:  56183579
canon:  56805283


## MPNN uniprot only

In [80]:
score_name = 'proteinmpnn_llr'
c = hl.read_table(f'gs://trisha-tmp/new_VSM_temp/{score_name}_collected.ht')
c = c.annotate(
    any_score = hl.find(lambda x: hl.is_defined(x[score_name]) == True, c.values)[score_name]

)
c_f = c.filter(hl.is_defined(c.any_score))
c_f.count()

Output()

Output()

Output()

63622442

In [84]:
c = c.annotate(
     canon_score = hl.find(lambda x: x.canonical == True, c.values)[score_name]
)

In [85]:
cf2 = c.filter((~hl.is_defined(c.canon_score)) & (hl.is_defined(c.any_score)))

In [86]:
cf2.show()

Output()

,,,,
locus,alleles,values,any_score,canon_score
locus<GRCh38>,array<str>,"array<struct{uniprot_em_id: str, aa_pos: int32, aa_ref: str, aa_alt: str, enst: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, transcript_mane_select: bool, uniprot_base: str, proteinmpnn_llr: float64}>",float64,float64
chr1:925945,"[""T"",""A""]","[(""A0A087WU74"",181,""S"",""T"",""ENST00000616016"",""SAMD11"",""ENSP00000478421"",""ENSG00000187634"",""NM_001385641.1"",True,True,True,NA,NA),(""A0A087WX24"",181,""S"",""T"",""ENST00000618323"",""SAMD11"",""ENSP00000480678"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087WXB3"",2,""S"",""T"",""ENST00000618181"",""SAMD11"",""ENSP00000480870"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087WYU3"",2,""S"",""T"",""ENST00000617307"",""SAMD11"",""ENSP00000482090"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087WYW1"",2,""S"",""T"",""ENST00000622503"",""SAMD11"",""ENSP00000482138"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087X1J5"",2,""S"",""T"",""ENST00000618779"",""SAMD11"",""ENSP00000484256"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087X223"",2,""S"",""T"",""ENST00000616125"",""SAMD11"",""ENSP00000484643"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""Q5SV95"",2,""S"",""T"",""ENST00000437963"",""SAMD11"",""ENSP00000393181"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""Q96NU1-3"",2,""S"",""T"",""ENST00000342066"",""SAMD11"",""ENSP00000342313"",""ENSG00000187634"",NA,False,False,False,""Q96NU1"",1.61e-02)]",1.61e-02,NA
chr1:925945,"[""T"",""C""]","[(""A0A087WU74"",181,""S"",""P"",""ENST00000616016"",""SAMD11"",""ENSP00000478421"",""ENSG00000187634"",""NM_001385641.1"",True,True,True,NA,NA),(""A0A087WX24"",181,""S"",""P"",""ENST00000618323"",""SAMD11"",""ENSP00000480678"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087WXB3"",2,""S"",""P"",""ENST00000618181"",""SAMD11"",""ENSP00000480870"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087WYU3"",2,""S"",""P"",""ENST00000617307"",""SAMD11"",""ENSP00000482090"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087WYW1"",2,""S"",""P"",""ENST00000622503"",""SAMD11"",""ENSP00000482138"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087X1J5"",2,""S"",""P"",""ENST00000618779"",""SAMD11"",""ENSP00000484256"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087X223"",2,""S"",""P"",""ENST00000616125"",""SAMD11"",""ENSP00000484643"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""Q5SV95"",2,""S"",""P"",""ENST00000437963"",""SAMD11"",""ENSP00000393181"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""Q96NU1-3"",2,""S"",""P"",""ENST00000342066"",""SAMD11"",""ENSP00000342313"",""ENSG00000187634"",NA,False,False,False,""Q96NU1"",-7.62e-01)]",-7.62e-01,NA
chr1:925945,"[""T"",""G""]","[(""A0A087WU74"",181,""S"",""A"",""ENST00000616016"",""SAMD11"",""ENSP00000478421"",""ENSG00000187634"",""NM_001385641.1"",True,True,True,NA,NA),(""A0A087WX24"",181,""S"",""A"",""ENST00000618323"",""SAMD11"",""ENSP00000480678"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087WXB3"",2,""S"",""A"",""ENST00000618181"",""SAMD11"",""ENSP00000480870"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087WYU3"",2,""S"",""A"",""ENST00000617307"",""SAMD11"",""ENSP00000482090"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087WYW1"",2,""S"",""A"",""ENST00000622503"",""SAMD11"",""ENSP00000482138"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087X1J5"",2,""S"",""A"",""ENST00000618779"",""SAMD11"",""ENSP00000484256"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""A0A087X223"",2,""S"",""A"",""ENST00000616125"",""SAMD11"",""ENSP00000484643"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""Q5SV95"",2,""S"",""A"",""ENST00000437963"",""SAMD11"",""ENSP00000393181"",""ENSG00000187634"",NA,False,False,False,NA,NA),(""Q96NU1-3"",2,""S"",""A"",""ENST00000342066"",""SAMD11"",""ENSP00000342313"",""ENSG00000187634"",NA,False,False,False,"

In [88]:
cf2_sm = cf2.head(5)

In [89]:
cf2_sm_e = cf2_sm.explode(cf2_sm.values)
cf2_sm_e.show()

Output()

+---------------+------------+----------------------+---------------+
| locus         | alleles    | values.uniprot_em_id | values.aa_pos |
+---------------+------------+----------------------+---------------+
| locus<GRCh38> | array<str> | str                  |         int32 |
+---------------+------------+----------------------+---------------+
| chr1:925945   | ["T","A"]  | "A0A087WU74"         |           181 |
| chr1:925945   | ["T","A"]  | "A0A087WX24"         |           181 |
| chr1:925945   | ["T","A"]  | "A0A087WXB3"         |             2 |
| chr1:925945   | ["T","A"]  | "A0A087WYU3"         |             2 |
| chr1:925945   | ["T","A"]  | "A0A087WYW1"         |             2 |
| chr1:925945   | ["T","A"]  | "A0A087X1J5"         |             2 |
| chr1:925945   | ["T","A"]  | "A0A087X223"         |             2 |
| chr1:925945   | ["T","A"]  | "Q5SV95"             |             2 |
| chr1:925945   | ["T","A"]  | "Q96NU1-3"           |             2 |
| chr1:925945   | ["T","C"]  | "A0A087WU74"         |           181 |
+---------------+------------+----------------------+---------------+

+---------------+---------------+-------------------+--------------------+
| values.aa_ref | values.aa_alt | values.enst       | values.gene_symbol |
+---------------+---------------+-------------------+--------------------+
| str           | str           | str               | str                |
+---------------+---------------+-------------------+--------------------+
| "S"           | "T"           | "ENST00000616016" | "SAMD11"           |
| "S"           | "T"           | "ENST00000618323" | "SAMD11"           |
| "S"           | "T"           | "ENST00000618181" | "SAMD11"           |
| "S"           | "T"           | "ENST00000617307" | "SAMD11"           |
| "S"           | "T"           | "ENST00000622503" | "SAMD11"           |
| "S"           | "T"           | "ENST00000618779" | "SAMD11"           |
| "S"           | "T"           | "ENST00000616125" | "SAMD11"           |
| "S"           | "T"           | "ENST00000437963" | "SAMD11"           |
| "S"           | "T"           | "ENST00000342066" | "SAMD11"           |
| "S"           | "P"           | "ENST00000616016" | "SAMD11"           |
+---------------+---------------+-------------------+--------------------+

+-------------------+-------------------+------------------+
| values.ensp       | values.ensg       | values.mane_NM   |
+-------------------+-------------------+------------------+
| str               | str               | str              |
+-------------------+-------------------+------------------+
| "ENSP00000478421" | "ENSG00000187634" | "NM_001385641.1" |
| "ENSP00000480678" | "ENSG00000187634" | NA               |
| "ENSP00000480870" | "ENSG00000187634" | NA               |
| "ENSP00000482090" | "ENSG00000187634" | NA               |
| "ENSP00000482138" | "ENSG00000187634" | NA               |
| "ENSP00000484256" | "ENSG00000187634" | NA               |
| "ENSP00000484643" | "ENSG00000187634" | NA               |
| "ENSP00000393181" | "ENSG00000187634" | NA               |
| "ENSP00000342313" | "ENSG00000187634" | NA               |
| "ENSP00000478421" | "ENSG00000187634" | "NM_001385641.1" |
+-------------------+-------------------+------------------+

+--------------------+------------------+-------------------------------+
| values.mane_select | values.canonical | values.transcript_mane_select |
+--------------------+------------------+-------------------------------+
|               bool |             bool |                          bool |
+--------------------+------------------+-------------------------------+
|               True |             True |                          True |
|              False |            False |                         False |
|              False |            False |                         False |
|              False |            False |                         False |
|              False |            F

In [90]:
cf2_sm_e.show(20)

Output()

+---------------+------------+----------------------+---------------+
| locus         | alleles    | values.uniprot_em_id | values.aa_pos |
+---------------+------------+----------------------+---------------+
| locus<GRCh38> | array<str> | str                  |         int32 |
+---------------+------------+----------------------+---------------+
| chr1:925945   | ["T","A"]  | "A0A087WU74"         |           181 |
| chr1:925945   | ["T","A"]  | "A0A087WX24"         |           181 |
| chr1:925945   | ["T","A"]  | "A0A087WXB3"         |             2 |
| chr1:925945   | ["T","A"]  | "A0A087WYU3"         |             2 |
| chr1:925945   | ["T","A"]  | "A0A087WYW1"         |             2 |
| chr1:925945   | ["T","A"]  | "A0A087X1J5"         |             2 |
| chr1:925945   | ["T","A"]  | "A0A087X223"         |             2 |
| chr1:925945   | ["T","A"]  | "Q5SV95"             |             2 |
| chr1:925945   | ["T","A"]  | "Q96NU1-3"           |             2 |
| chr1:925945   | ["T","C"]  | "A0A087WU74"         |           181 |
| chr1:925945   | ["T","C"]  | "A0A087WX24"         |           181 |
| chr1:925945   | ["T","C"]  | "A0A087WXB3"         |             2 |
| chr1:925945   | ["T","C"]  | "A0A087WYU3"         |             2 |
| chr1:925945   | ["T","C"]  | "A0A087WYW1"         |             2 |
| chr1:925945   | ["T","C"]  | "A0A087X1J5"         |             2 |
| chr1:925945   | ["T","C"]  | "A0A087X223"         |             2 |
| chr1:925945   | ["T","C"]  | "Q5SV95"             |             2 |
| chr1:925945   | ["T","C"]  | "Q96NU1-3"           |             2 |
| chr1:925945   | ["T","G"]  | "A0A087WU74"         |           181 |
| chr1:925945   | ["T","G"]  | "A0A087WX24"         |           181 |
+---------------+------------+----------------------+---------------+

+---------------+---------------+-------------------+--------------------+
| values.aa_ref | values.aa_alt | values.enst       | values.gene_symbol |
+---------------+---------------+-------------------+--------------------+
| str           | str           | str               | str                |
+---------------+---------------+-------------------+--------------------+
| "S"           | "T"           | "ENST00000616016" | "SAMD11"           |
| "S"           | "T"           | "ENST00000618323" | "SAMD11"           |
| "S"           | "T"           | "ENST00000618181" | "SAMD11"           |
| "S"           | "T"           | "ENST00000617307" | "SAMD11"           |
| "S"           | "T"           | "ENST00000622503" | "SAMD11"           |
| "S"           | "T"           | "ENST00000618779" | "SAMD11"           |
| "S"           | "T"           | "ENST00000616125" | "SAMD11"           |
| "S"           | "T"           | "ENST00000437963" | "SAMD11"           |
| "S"           | "T"           | "ENST00000342066" | "SAMD11"           |
| "S"           | "P"           | "ENST00000616016" | "SAMD11"           |
| "S"           | "P"           | "ENST00000618323" | "SAMD11"           |
| "S"           | "P"           | "ENST00000618181" | "SAMD11"           |
| "S"           | "P"           | "ENST00000617307" | "SAMD11"           |
| "S"           | "P"           | "ENST00000622503" | "SAMD11"           |
| "S"           | "P"           | "ENST00000618779" | "SAMD11"           |
| "S"           | "P"           | "ENST00000616125" | "SAMD11"           |
| "S"           | "P"           | "ENST00000437963" | "SAMD11"           |
| "S"           | "P"           | "ENST00000342066" | "SAMD11"           |
| "S"           | "A"           | "ENST00000616016" | "SAMD11"           |
| "S"           | "A"           | "ENST00000618323" | "SAMD11"           |
+---------------+---------------+-------------------+--------------------+

+-------------------+-------------------+------------------+
| values.ensp       | values.ensg       | values.mane_NM   |
+-------------------+-------------------+------------------+
| str               | str               | str

# PAI3D


In [67]:
vsm_ht = hl.read_table('gs://trisha-tmp/make_linker/linker2_temp_union_final.ht')
vsm_ht = vsm_ht.annotate(
    uniprot_base = hl.if_else(
        vsm_ht.uniprot_em_id.contains('-'), vsm_ht.uniprot_em_id.split('-')[0], hl.missing(hl.tstr)
    )
)
vsm_ht = vsm_ht.key_by('locus', 'alleles', 'enst')


Output()

Output()

In [68]:
ht = hl.read_table(vsm_ens['PAI3D']['path'])
score_name = vsm_ens['PAI3D']['score_cols'][0]
# ht is already keyed by locus, allele, enst
vsm_ht = vsm_ht.join(ht, how='left')

Output()

Output()

In [69]:
mane_count, canon_count = check_score_completenes(vsm_ht, score_name)
print('mane: ', mane_count)
print('canon: ', canon_count)

Output()

Output()

Output()

mane:  51923773
canon:  53265688


In [70]:
vsm_ht.write('gs://trisha-tmp/new_VSM_temp/vsm_pai3d.ht', overwrite=True)

Output()

In [75]:
c = hl.read_table(f'gs://trisha-tmp/new_VSM_temp/{score_name}_collected.ht')
c = c.annotate(
    any_score = hl.find(lambda x: hl.is_defined(x[score_name]) == True, c.values)[score_name]

)

Output()

Output()

In [76]:
c_f = c.filter(hl.is_defined(c.any_score))
c_f.count()

Output()

68937617

In [78]:
c.show()

Output()

,,,
locus,alleles,values,any_score
locus<GRCh38>,array<str>,"array<struct{enst: str, aa_pos: int32, aa_ref: str, aa_alt: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str, score_PAI3D: float64}>",float64
chr1:65568,"[""A"",""C""]","[(""ENST00000641515"",2,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA)]",NA
chr1:65568,"[""A"",""G""]","[(""ENST00000641515"",2,""K"",""E"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA)]",NA
chr1:65569,"[""A"",""C""]","[(""ENST00000641515"",2,""K"",""T"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA)]",NA
chr1:65569,"[""A"",""G""]","[(""ENST00000641515"",2,""K"",""R"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA)]",NA
chr1:65569,"[""A"",""T""]","[(""ENST00000641515"",2,""K"",""M"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA)]",NA
chr1:65570,"[""G"",""C""]","[(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA)]",NA
chr1:65570,"[""G"",""T""]","[(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA)]",NA
chr1:65571,"[""A"",""C""]","[(""ENST00000641515"",3,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA)]",NA


In [79]:
c_f.show()

Output()

Received a keyboard interrupt, cancelling the batch...
execute(...): ]8;id=171321;https://batch.hail.is/batches/8323386\8323386]8;;\ ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% 0/5 -:--:-- 0:09:42

KeyboardInterrupt: 

# RASP

In [91]:
rasp = hl.read_table('gs://missense-scoring/mutation/rasp_scores.ht')


Output()

Output()

In [92]:
rasp.show()

Output()

,,,,
uniprot_id,uniprot_aa_pos,uniprot_aa_ref,uniprot_aa_alt,rasp_score
str,int32,str,str,float64
"""A0A024R1R8""",1,"""M""","""A""",5.34e-02
"""A0A024R1R8""",1,"""M""","""C""",5.60e-01
"""A0A024R1R8""",1,"""M""","""D""",-2.04e-01
"""A0A024R1R8""",1,"""M""","""E""",2.17e-01
"""A0A024R1R8""",1,"""M""","""F""",7.70e-01
"""A0A024R1R8""",1,"""M""","""G""",-2.19e-01
"""A0A024R1R8""",1,"""M""","""H""",2.49e-01
"""A0A024R1R8""",1,"""M""","""I""",1.22e-01


In [93]:
af_ht = hl.read_table('gs://genetics-gym-not-public/Emily/af2db_uniprot_final.ht')


Output()

Output()

In [96]:
rasp = rasp.key_by('uniprot_id')
rasp = rasp.join(af_ht, how='left')

In [100]:
rasp.show()

Output()

,,,,,
uniprot_id,aa_pos,aa_ref,aa_alt,rasp_score,uniprot_isoform
str,int32,str,str,float64,str
"""A0A024R1R8""",1,"""M""","""A""",5.34e-02,"""A0A024R1R8"""
"""A0A024R1R8""",1,"""M""","""C""",5.60e-01,"""A0A024R1R8"""
"""A0A024R1R8""",1,"""M""","""D""",-2.04e-01,"""A0A024R1R8"""
"""A0A024R1R8""",1,"""M""","""E""",2.17e-01,"""A0A024R1R8"""
"""A0A024R1R8""",1,"""M""","""F""",7.70e-01,"""A0A024R1R8"""
"""A0A024R1R8""",1,"""M""","""G""",-2.19e-01,"""A0A024R1R8"""
"""A0A024R1R8""",1,"""M""","""H""",2.49e-01,"""A0A024R1R8"""
"""A0A024R1R8""",1,"""M""","""I""",1.22e-01,"""A0A024R1R8"""


In [101]:
# rasp = rasp.rename({'uniprot_aa_pos': 'aa_pos', 'uniprot_aa_ref': 'aa_ref', 'uniprot_aa_alt': 'aa_alt'})
rasp_k = rasp.key_by('uniprot_isoform', 'aa_pos', 'aa_ref', 'aa_alt')
rasp_k = rasp_k.select(rasp_k.rasp_score)


In [102]:
rasp_k.show()

Output()

,,,,
uniprot_isoform,aa_pos,aa_ref,aa_alt,rasp_score
str,int32,str,str,float64
"""A0A024R1R8""",1,"""M""","""A""",5.34e-02
"""A0A024R1R8""",1,"""M""","""C""",5.60e-01
"""A0A024R1R8""",1,"""M""","""D""",-2.04e-01
"""A0A024R1R8""",1,"""M""","""E""",2.17e-01
"""A0A024R1R8""",1,"""M""","""F""",7.70e-01
"""A0A024R1R8""",1,"""M""","""G""",-2.19e-01
"""A0A024R1R8""",1,"""M""","""H""",2.49e-01
"""A0A024R1R8""",1,"""M""","""I""",1.22e-01


In [104]:
vsm_ht = hl.read_table('gs://trisha-tmp/new_VSM_temp/vsm_uniprot_iso_keyed.ht')
vsm_ht = vsm_ht.join(rasp_k, how='left')

Output()

Output()

In [106]:
results = check_score_completenes(vsm_ht, 'rasp_score')
print(results)

Output()

Output()

Output()

(59553001, 60198530)


In [111]:
score_name = 'rasp_score'
check_ht = hl.read_table(f'gs://trisha-tmp/new_VSM_temp/rasp_score_collected.ht')
check_ht = check_ht.annotate(
    score_mane = hl.find(lambda x: x.mane_select == True, check_ht.values)[score_name],
    is_mane = hl.find(lambda x: x.mane_select == True, check_ht.values)['mane_select'],
    score_canon =  hl.find(lambda x: x.canonical == True, check_ht.values)[score_name],
    is_canonical = hl.find(lambda x: x.canonical == True, check_ht.values)['canonical'],
    score_any = hl.find(lambda x: hl.is_defined(x[score_name]) == True, check_ht.values)[score_name]

)  
check_htf1 = check_ht.filter(hl.is_defined(check_ht.score_mane))
check_htf2 = check_ht.filter(hl.is_defined(check_ht.score_canon))
check_htf3 = check_ht.filter(hl.is_defined(check_ht.score_any))
mane_count = check_htf1.count()
canon_count = check_htf2.count()
any_count = check_htf3.count()
print(any_count)
count_results = {'mane': mane_count, 'canon': canon_count, 'any': any_count}
print(count_results)

Output()

Output()

Output()

Output()

Output()

67825466
{'mane': 59553001, 'canon': 60198530, 'any': 67825466}


# Popeve

In [8]:
popeve_raw_ht = hl.import_table(
    'gs://missense-scoring/popEVE_ukbb_20250312/*.csv', # 2025 popeve scores
    delimiter=',',
    source_file_field='filename',
    types = {
        'popEVE':hl.tfloat,
        'popped EVE':hl.tfloat,
        'popped ESM-1v':hl.tfloat,
        'EVE':hl.tfloat,
        'ESM-1v':hl.tfloat,
        
    },
    force=True,
    missing='',
)

Output()

In [9]:
popeve_raw_ht.show()

Output()

,,,,,,,
mutant,gap frequency,popEVE,popped EVE,popped ESM-1v,EVE,ESM-1v,filename
str,str,float64,float64,float64,float64,float64,str
"""M1A""","""0.846""",-4.48e+00,-4.71e+00,-4.25e+00,1.32e+01,-6.11e+00,"""gs://missense-scoring/popEVE_ukbb_20250312/NP_000005.3.csv"""
"""M1C""","""0.846""",-4.57e+00,-4.70e+00,-4.44e+00,1.31e+01,-7.16e+00,"""gs://missense-scoring/popEVE_ukbb_20250312/NP_000005.3.csv"""
"""M1D""","""0.846""",-4.50e+00,-4.54e+00,-4.45e+00,1.22e+01,-7.23e+00,"""gs://missense-scoring/popEVE_ukbb_20250312/NP_000005.3.csv"""
"""M1E""","""0.846""",-4.45e+00,-4.52e+00,-4.38e+00,1.21e+01,-6.82e+00,"""gs://missense-scoring/popEVE_ukbb_20250312/NP_000005.3.csv"""
"""M1F""","""0.846""",-4.46e+00,-4.56e+00,-4.37e+00,1.23e+01,-6.76e+00,"""gs://missense-scoring/popEVE_ukbb_20250312/NP_000005.3.csv"""
"""M1G""","""0.846""",-4.39e+00,-4.50e+00,-4.29e+00,1.19e+01,-6.34e+00,"""gs://missense-scoring/popEVE_ukbb_20250312/NP_000005.3.csv"""
"""M1H""","""0.846""",-4.43e+00,-4.43e+00,-4.43e+00,1.15e+01,-7.11e+00,"""gs://missense-scoring/popEVE_ukbb_20250312/NP_000005.3.csv"""
"""M1I""","""0.846""",-4.42e+00,-4.56e+00,-4.28e+00,1.23e+01,-6.31e+00,"""gs://missense-scoring/popEVE_ukbb_20250312/NP_000005.3.csv"""


# Misfit

In [10]:
vsm_misfit = hl.read_table('gs://trisha-tmp/new_VSM_temp/vsm_misfit.ht')

Output()

Output()

In [11]:
vsm_misfit.show()

Output()

,,,,,,,,,,,,,,,,,,,,
enst,aa_pos,aa_alt,locus,alleles,aa_ref,gene_symbol,ensp,ensg,mane_NM,mane_select,canonical,uniprot_em_id,transcript_mane_select,uniprot_base,MisFit_D,MisFit_S,uniprot_id,ensg_1,ensp_1,gene_symbol_1
str,int32,str,locus<GRCh38>,array<str>,str,str,str,str,str,bool,bool,str,bool,str,float64,float64,str,str,str,str
"""ENST00000000233""",2,"""A""",chr7:127588503,"[""G"",""C""]","""G""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA,7.90e-01,3.31e-02,"""P84085""","""ENSG00000004059""","""ENSP00000000233""","""ARF5"""
"""ENST00000000233""",2,"""C""",chr7:127588502,"[""G"",""T""]","""G""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA,8.91e-01,6.75e-02,"""P84085""","""ENSG00000004059""","""ENSP00000000233""","""ARF5"""
"""ENST00000000233""",2,"""D""",chr7:127588503,"[""G"",""A""]","""G""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA,8.42e-01,4.80e-02,"""P84085""","""ENSG00000004059""","""ENSP00000000233""","""ARF5"""
"""ENST00000000233""",2,"""R""",chr7:127588502,"[""G"",""C""]","""G""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA,9.12e-01,7.81e-02,"""P84085""","""ENSG00000004059""","""ENSP00000000233""","""ARF5"""
"""ENST00000000233""",2,"""S""",chr7:127588502,"[""G"",""A""]","""G""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA,8.36e-01,4.61e-02,"""P84085""","""ENSG00000004059""","""ENSP00000000233""","""ARF5"""
"""ENST00000000233""",2,"""V""",chr7:127588503,"[""G"",""T""]","""G""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA,9.09e-01,7.66e-02,"""P84085""","""ENSG00000004059""","""ENSP00000000233""","""ARF5"""
"""ENST00000000233""",3,"""F""",chr7:127588505,"[""C"",""T""]","""L""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA,5.51e-01,5.83e-03,"""P84085""","""ENSG00000004059""","""ENSP00000000233""","""ARF5"""
"""ENST00000000233""",3,"""H""",chr7:127588506,"[""T"",""A""]","""L""","""ARF5""","""ENSP00000000233""","""ENSG00000004059""","""NM_001662.4""",True,True,"""P84085""",True,NA,5.96e-01,8.09e-03,"""P84085""","""ENSG00000004059""","""ENSP00000000233""","""ARF5"""


In [13]:
misfit_collected = hl.read_table('gs://trisha-tmp/new_VSM_temp/MisFit_D_collected.ht')

Output()

Output()

In [15]:
score_name = 'MisFit_D'
misfit_collected = misfit_collected.annotate(
        score_mane = hl.find(lambda x: x.mane_select == True, misfit_collected.values)[score_name],
        is_mane = hl.find(lambda x: x.mane_select == True, misfit_collected.values)['mane_select'],
        score_canon =  hl.find(lambda x: x.canonical == True, misfit_collected.values)[score_name],
        is_canonical = hl.find(lambda x: x.canonical == True, misfit_collected.values)['canonical'],
        score_any = hl.find(lambda x: hl.is_defined(x[score_name]) == True, misfit_collected.values)[score_name]

    )  

In [16]:
misfit_collected.show()

Output()

,,,,,,,
locus,alleles,values,score_mane,is_mane,score_canon,is_canonical,score_any
locus<GRCh38>,array<str>,"array<struct{enst: str, aa_pos: int32, aa_alt: str, aa_ref: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str, MisFit_D: float64, MisFit_S: float64, uniprot_id: str, ensg_1: str, ensp_1: str, gene_symbol_1: str}>",float64,bool,float64,bool,float64
chr1:65568,"[""A"",""C""]","[(""ENST00000641515"",2,""Q"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65568,"[""A"",""G""]","[(""ENST00000641515"",2,""E"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65569,"[""A"",""C""]","[(""ENST00000641515"",2,""T"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65569,"[""A"",""G""]","[(""ENST00000641515"",2,""R"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65569,"[""A"",""T""]","[(""ENST00000641515"",2,""M"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65570,"[""G"",""C""]","[(""ENST00000641515"",2,""N"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65570,"[""G"",""T""]","[(""ENST00000641515"",2,""N"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65571,"[""A"",""C""]","[(""ENST00000641515"",3,""Q"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA


In [20]:
misfit_collected_f = misfit_collected.filter(~hl.is_defined(misfit_collected.score_mane))
misfit_collected_f.show()

Output()

,,,,,,,
locus,alleles,values,score_mane,is_mane,score_canon,is_canonical,score_any
locus<GRCh38>,array<str>,"array<struct{enst: str, aa_pos: int32, aa_alt: str, aa_ref: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str, MisFit_D: float64, MisFit_S: float64, uniprot_id: str, ensg_1: str, ensp_1: str, gene_symbol_1: str}>",float64,bool,float64,bool,float64
chr1:65568,"[""A"",""C""]","[(""ENST00000641515"",2,""Q"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65568,"[""A"",""G""]","[(""ENST00000641515"",2,""E"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65569,"[""A"",""C""]","[(""ENST00000641515"",2,""T"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65569,"[""A"",""G""]","[(""ENST00000641515"",2,""R"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65569,"[""A"",""T""]","[(""ENST00000641515"",2,""M"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65570,"[""G"",""C""]","[(""ENST00000641515"",2,""N"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65570,"[""G"",""T""]","[(""ENST00000641515"",2,""N"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA
chr1:65571,"[""A"",""C""]","[(""ENST00000641515"",3,""Q"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA


In [21]:
misfit_collected_f = misfit_collected_f.annotate(
    mane_enst = hl.find(lambda x: x.mane_select == True, misfit_collected_f.values)['enst'], 
    uniprot = hl.find(lambda x: x.mane_select == True, misfit_collected_f.values)['uniprot_em_id']
)

In [22]:
misfit_collected_f.show()

Output()

,,,,,,,,,
locus,alleles,values,score_mane,is_mane,score_canon,is_canonical,score_any,mane_enst,uniprot
locus<GRCh38>,array<str>,"array<struct{enst: str, aa_pos: int32, aa_alt: str, aa_ref: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str, MisFit_D: float64, MisFit_S: float64, uniprot_id: str, ensg_1: str, ensp_1: str, gene_symbol_1: str}>",float64,bool,float64,bool,float64,str,str
chr1:65568,"[""A"",""C""]","[(""ENST00000641515"",2,""Q"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA,"""ENST00000641515""","""A0A2U3U0J3"""
chr1:65568,"[""A"",""G""]","[(""ENST00000641515"",2,""E"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA,"""ENST00000641515""","""A0A2U3U0J3"""
chr1:65569,"[""A"",""C""]","[(""ENST00000641515"",2,""T"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA,"""ENST00000641515""","""A0A2U3U0J3"""
chr1:65569,"[""A"",""G""]","[(""ENST00000641515"",2,""R"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA,"""ENST00000641515""","""A0A2U3U0J3"""
chr1:65569,"[""A"",""T""]","[(""ENST00000641515"",2,""M"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA,"""ENST00000641515""","""A0A2U3U0J3"""
chr1:65570,"[""G"",""C""]","[(""ENST00000641515"",2,""N"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA,"""ENST00000641515""","""A0A2U3U0J3"""
chr1:65570,"[""G"",""T""]","[(""ENST00000641515"",2,""N"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA,"""ENST00000641515""","""A0A2U3U0J3"""
chr1:65571,"[""A"",""C""]","[(""ENST00000641515"",3,""Q"",""K"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA,NA,NA,NA,NA,NA,NA)]",NA,True,NA,True,NA,"""ENST00000641515""","""A0A2U3U0J3"""


In [23]:
uniprot_missing = misfit_collected_f.aggregate(hl.agg.collect_as_set(misfit_collected_f.uniprot))

Output()

In [24]:
len(uniprot_missing)

1873

In [25]:
uniprot_missing


{'O60224-1',
 'A0A1R3UHJ7',
 'Q03721-3',
 'Q7Z5Q1-9',
 'Q8N257',
 'Q13620-1',
 'A0A8I5KQI9',
 'A0A804HIA4',
 'A8K830-5',
 'Q86SG7-1',
 'A0A0A0MQZ0',
 'Q9UQB8-2',
 'Q8N9C0-2',
 'G5EA03',
 'P28332-2',
 'Q9NVX7-2',
 'P57078-2',
 'Q9UBM8-1',
 'A0A1W2PQW8',
 'A0A804HL01',
 'A0A7P0T7G7',
 'Q9H0B3-4',
 'E9PCW1',
 'Q96AA8-3',
 'A8MV23-2',
 'P22626-2',
 'Q96GU1-2',
 'A0A0U1RRK5',
 'A0A087WU74',
 'J3QT39',
 'G3XAK1',
 'P09496-2',
 'Q8N9Z9-5',
 'Q5VZM2-2',
 'P35346',
 'Q8WXK4-2',
 'J3KQ40',
 'Q149M9-3',
 'Q15149-4',
 'P55075-4',
 'Q5VT25-2',
 'A6NK59-3',
 'Q9NZH5-2',
 'A6NC05-2',
 'A0A0C4DFM0',
 'P60896-1',
 'Q9UBX7-1',
 'Q9ULG6-5',
 'O15079-2',
 'O43734-2',
 'Q8N2N9-1',
 'Q86VQ3-2',
 'Q9Y6D0',
 'E9PFI2',
 'A0A087WXV5',
 'Q8ND56-2',
 'O15027-1',
 'P15622-3',
 'P57103-2',
 'P43626-1',
 'Q9Y3Y4-2',
 'Q9UKR8-2',
 'Q9BRK0-2',
 'Q86W92-2',
 'Q08174-2',
 'A0A0A0MS48',
 'Q9UI36-2',
 'B2RXH4',
 'Q8IU80-1',
 'A0A6I8PRB2',
 'A0A0A0MR89',
 'A0A7I2UT39',
 'Q9NQ33',
 'A0A0C4DGS1',
 'A0A087X2D5',
 'J3KNZ9',
 '

In [18]:
misfit_collected_sm_exp = misfit_collected_sm.explode(misfit_collected_sm.values)

In [19]:
misfit_collected_sm_exp.show()

Output()

+---------------+------------+-------------------+---------------+
| locus         | alleles    | values.enst       | values.aa_pos |
+---------------+------------+-------------------+---------------+
| locus<GRCh38> | array<str> | str               |         int32 |
+---------------+------------+-------------------+---------------+
| chr1:65568    | ["A","C"]  | "ENST00000641515" |             2 |
| chr1:65568    | ["A","G"]  | "ENST00000641515" |             2 |
| chr1:65569    | ["A","C"]  | "ENST00000641515" |             2 |
| chr1:65569    | ["A","G"]  | "ENST00000641515" |             2 |
| chr1:65569    | ["A","T"]  | "ENST00000641515" |             2 |
+---------------+------------+-------------------+---------------+

+---------------+---------------+--------------------+-------------------+
| values.aa_alt | values.aa_ref | values.gene_symbol | values.ensp       |
+---------------+---------------+--------------------+-------------------+
| str           | str           | str                | str               |
+---------------+---------------+--------------------+-------------------+
| "Q"           | "K"           | "OR4F5"            | "ENSP00000493376" |
| "E"           | "K"           | "OR4F5"            | "ENSP00000493376" |
| "T"           | "K"           | "OR4F5"            | "ENSP00000493376" |
| "R"           | "K"           | "OR4F5"            | "ENSP00000493376" |
| "M"           | "K"           | "OR4F5"            | "ENSP00000493376" |
+---------------+---------------+--------------------+-------------------+

+-------------------+------------------+--------------------+------------------+
| values.ensg       | values.mane_NM   | values.mane_select | values.canonical |
+-------------------+------------------+--------------------+------------------+
| str               | str              |               bool |             bool |
+-------------------+------------------+--------------------+------------------+
| "ENSG00000186092" | "NM_001005484.2" |               True |             True |
| "ENSG00000186092" | "NM_001005484.2" |               True |             True |
| "ENSG00000186092" | "NM_001005484.2" |               True |             True |
| "ENSG00000186092" | "NM_001005484.2" |               True |             True |
| "ENSG00000186092" | "NM_001005484.2" |               True |             True |
+-------------------+------------------+--------------------+------------------+

+----------------------+-------------------------------+---------------------+
| values.uniprot_em_id | values.transcript_mane_select | values.uniprot_base |
+----------------------+-------------------------------+---------------------+
| str                  |                          bool | str                 |
+----------------------+-------------------------------+---------------------+
| "A0A2U3U0J3"         |                          True | NA                  |
| "A0A2U3U0J3"         |                          True | NA                  |
| "A0A2U3U0J3"         |                          True | NA                  |
| "A0A2U3U0J3"         |                          True | NA                  |
| "A0A2U3U0J3"         |                          True | NA                  |
+----------------------+-------------------------------+---------------------+

+-----------------+-----------------+-------------------+---------------+
| values.MisFit_D | values.MisFit_S | values.uniprot_id | values.ensg_1 |
+-----------------+-----------------+-------------------+---------------+
|         float64 |         float64 | str               | str           |
+-----------------+-----------------+-------------------+---------------+
|              NA |              NA | NA                | NA            |
|              NA |              NA | NA                | NA            |
|              NA |              NA | NA                | NA            |
|              NA |              NA | NA                | NA            |
|       

In [26]:
cpt_ht = hl.import_table('gs://missense-scoring/cpt_all/*.csv.gz', 
                        delimiter=',', force=True, source_file_field='filename')
cpt_ht = cpt_ht.annotate(
    aa_pos = hl.int(cpt_ht.mutant[1:-1]),
    aa_ref = cpt_ht.mutant[0],
    aa_alt = cpt_ht.mutant[-1],
    cpt_score = hl.float(cpt_ht.CPT1_score), 
    uniprot_entry = cpt_ht.filename.split('/')[-1].split('\.')[0]
)

Output()

In [27]:
cpt_ht.show()

Output()

,,,,,,,
mutant,CPT1_score,filename,aa_pos,aa_ref,aa_alt,cpt_score,uniprot_entry
str,str,str,int32,str,str,float64,str
"""M1A""","""0.29787490035462005""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""A""",2.98e-01,"""1433B_HUMAN"""
"""M1R""","""0.25329334807909964""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""R""",2.53e-01,"""1433B_HUMAN"""
"""M1N""","""0.4805691574686546""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""N""",4.81e-01,"""1433B_HUMAN"""
"""M1D""","""0.488481879797412""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""D""",4.88e-01,"""1433B_HUMAN"""
"""M1C""","""0.45939387212492444""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""C""",4.59e-01,"""1433B_HUMAN"""
"""M1Q""","""0.4017759248638677""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""Q""",4.02e-01,"""1433B_HUMAN"""
"""M1E""","""0.46986664927866606""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""E""",4.70e-01,"""1433B_HUMAN"""
"""M1G""","""0.351865753584384""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""G""",3.52e-01,"""1433B_HUMAN"""


In [28]:
uniprot_mapping = hl.import_table('gs://genetics-gym-not-public/Trisha/CPT_entry_to_accession.tsv', delimiter='\t')

Output()

In [29]:
uniprot_mapping.show()

Output()

,
entry_name,accession
str,str
"""1433B_HUMAN""","""P31946"""
"""1433E_HUMAN""","""P62258"""
"""1433F_HUMAN""","""Q04917"""
"""1433G_HUMAN""","""P61981"""
"""1433S_HUMAN""","""P31947"""
"""1433T_HUMAN""","""P27348"""
"""1433Z_HUMAN""","""P63104"""
"""1A1L1_HUMAN""","""Q96QU6"""


In [30]:
uniprot_mapping_k = uniprot_mapping.key_by('entry_name')

In [31]:
cpt_ht = cpt_ht.annotate(
    uniprot_id = uniprot_mapping_k[cpt_ht.uniprot_entry].accession
)

In [32]:
cpt_ht.show()

Output()

,,,,,,,,
mutant,CPT1_score,filename,aa_pos,aa_ref,aa_alt,cpt_score,uniprot_entry,uniprot_id
str,str,str,int32,str,str,float64,str,str
"""M1A""","""0.29787490035462005""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""A""",2.98e-01,"""1433B_HUMAN""","""P31946"""
"""M1R""","""0.25329334807909964""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""R""",2.53e-01,"""1433B_HUMAN""","""P31946"""
"""M1N""","""0.4805691574686546""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""N""",4.81e-01,"""1433B_HUMAN""","""P31946"""
"""M1D""","""0.488481879797412""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""D""",4.88e-01,"""1433B_HUMAN""","""P31946"""
"""M1C""","""0.45939387212492444""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""C""",4.59e-01,"""1433B_HUMAN""","""P31946"""
"""M1Q""","""0.4017759248638677""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""Q""",4.02e-01,"""1433B_HUMAN""","""P31946"""
"""M1E""","""0.46986664927866606""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""E""",4.70e-01,"""1433B_HUMAN""","""P31946"""
"""M1G""","""0.351865753584384""","""gs://missense-scoring/cpt_all/1433B_HUMAN.csv.gz""",1,"""M""","""G""",3.52e-01,"""1433B_HUMAN""","""P31946"""


# finish ESM counts

In [35]:
esm_collected = hl.read_table('gs://trisha-tmp/new_VSM_temp/esm1b_lj2_collected.ht')

Output()

Output()

In [36]:
esm_collected.show()

A transient error occured. We will automatically retry. Do not be alarmed. We have thus far seen 2 transient errors (next delay: 3.033s). The most recent error was <class 'hailtop.httpx.ClientResponseError'> 503, message='Service Unavailable', url=URL('https://batch.hail.is/api/v1alpha/batches/8324924/update-fast') body='upstream connect error or disconnect/reset before headers. reset reason: connection failure'. 


Output()

,,,,
locus,alleles,values,mane_select,esm_c
locus<GRCh38>,array<str>,"array<struct{uniprot_base: str, aa_pos: int32, aa_ref: str, aa_alt: str, uniprot_em_id: str, enst: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, transcript_mane_select: bool, esm_score: float64, esm_score_1: float64, esm_comb: float64}>",array<bool>,array<float64>
chr1:65568,"[""A"",""C""]","[(NA,2,""K"",""Q"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65568,"[""A"",""G""]","[(NA,2,""K"",""E"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65569,"[""A"",""C""]","[(NA,2,""K"",""T"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65569,"[""A"",""G""]","[(NA,2,""K"",""R"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65569,"[""A"",""T""]","[(NA,2,""K"",""M"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65570,"[""G"",""C""]","[(NA,2,""K"",""N"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65570,"[""G"",""T""]","[(NA,2,""K"",""N"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]
chr1:65571,"[""A"",""C""]","[(NA,3,""K"",""Q"",""A0A2U3U0J3"",""ENST00000641515"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,True,NA,NA,NA)]",[True],[NA]


In [37]:
esm_collected2 = esm_collected.drop('mane_select', 'esm_c')

In [40]:
esm_collected2.describe()

----------------------------------------
Global fields:
    'grange': array<int32> 
    'vep_help': str 
    'vep_config': str 
    'version': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'values': array<struct {
        uniprot_base: str, 
        aa_pos: int32, 
        aa_ref: str, 
        aa_alt: str, 
        uniprot_em_id: str, 
        enst: str, 
        gene_symbol: str, 
        ensp: str, 
        ensg: str, 
        mane_NM: str, 
        mane_select: bool, 
        canonical: bool, 
        transcript_mane_select: bool, 
        esm_score: float64, 
        esm_score_1: float64, 
        esm_comb: float64
    }> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [41]:
results = check_score_completenes(esm_collected, 'esm_comb')
results

Output()

Output()

Output()

{'mane': 65780956, 'canon': 67279688, 'any': 72281380}

# Check popeve

In [42]:
popeve_vsm = hl.read_table('gs://trisha-tmp/new_VSM_temp/vsm_popeve.ht')

Output()

Output()

In [43]:
popeve_vsm.show()


Output()

,,,,,,,,,,,,,,,,
NP,aa_pos,aa_ref,aa_alt,locus,alleles,gene_symbol,NM,ncbi,mane_ENST,mane_select,canonical,popEVE,popped_EVE,popped_ESM_1v,EVE,ESM_1v
str,int32,str,str,locus<GRCh38>,array<str>,str,str,str,str,bool,bool,float64,float64,float64,float64,float64
"""NP_000005.3""",2,"""G""","""A""",chr12:9115845,"[""C"",""G""]","""A2M""","""NM_000014.6""","""2""","""ENST00000318602.12""",True,True,-3.09e+00,-3.02e+00,-3.16e+00,3.30e+00,5.61e-01
"""NP_000005.3""",2,"""G""","""E""",chr12:9115845,"[""C"",""T""]","""A2M""","""NM_000014.6""","""2""","""ENST00000318602.12""",True,True,-3.17e+00,-3.14e+00,-3.20e+00,4.01e+00,2.57e-01
"""NP_000005.3""",2,"""G""","""R""",chr12:9115846,"[""C"",""G""]","""A2M""","""NM_000014.6""","""2""","""ENST00000318602.12""",True,True,-3.02e+00,-2.73e+00,-3.31e+00,1.64e+00,-6.00e-01
"""NP_000005.3""",2,"""G""","""R""",chr12:9115846,"[""C"",""T""]","""A2M""","""NM_000014.6""","""2""","""ENST00000318602.12""",True,True,-3.02e+00,-2.73e+00,-3.31e+00,1.64e+00,-6.00e-01
"""NP_000005.3""",2,"""G""","""V""",chr12:9115845,"[""C"",""A""]","""A2M""","""NM_000014.6""","""2""","""ENST00000318602.12""",True,True,-3.41e+00,-3.51e+00,-3.32e+00,6.15e+00,-6.34e-01
"""NP_000005.3""",2,"""G""","""W""",chr12:9115846,"[""C"",""A""]","""A2M""","""NM_000014.6""","""2""","""ENST00000318602.12""",True,True,-3.31e+00,-3.20e+00,-3.42e+00,4.35e+00,-1.31e+00
"""NP_000005.3""",3,"""K""","""E""",chr12:9115843,"[""T"",""C""]","""A2M""","""NM_000014.6""","""2""","""ENST00000318602.12""",True,True,-3.23e+00,-3.17e+00,-3.29e+00,4.21e+00,-4.05e-01
"""NP_000005.3""",3,"""K""","""M""",chr12:9115842,"[""T"",""A""]","""A2M""","""NM_000014.6""","""2""","""ENST00000318602.12""",True,True,-3.46e+00,-3.63e+00,-3.29e+00,6.88e+00,-3.78e-01


In [44]:
popeve_vsmf = popeve_vsm.filter((~hl.is_defined(popeve_vsm.popEVE)) & (popeve_vsm.mane_select == True))

In [45]:
popeve_vsmf.show()


Output()

,,,,,,,,,,,,,,,,
NP,aa_pos,aa_ref,aa_alt,locus,alleles,gene_symbol,NM,ncbi,mane_ENST,mane_select,canonical,popEVE,popped_EVE,popped_ESM_1v,EVE,ESM_1v
str,int32,str,str,locus<GRCh38>,array<str>,str,str,str,str,bool,bool,float64,float64,float64,float64,float64
"""NP_000029.2""",2,"""A""","""D""",chr5:112754895,"[""C"",""A""]","""APC""","""NM_000038.6""","""324""","""ENST00000257430.9""",True,True,NA,NA,NA,NA,NA
"""NP_000029.2""",2,"""A""","""G""",chr5:112754895,"[""C"",""G""]","""APC""","""NM_000038.6""","""324""","""ENST00000257430.9""",True,True,NA,NA,NA,NA,NA
"""NP_000029.2""",2,"""A""","""P""",chr5:112754894,"[""G"",""C""]","""APC""","""NM_000038.6""","""324""","""ENST00000257430.9""",True,True,NA,NA,NA,NA,NA
"""NP_000029.2""",2,"""A""","""S""",chr5:112754894,"[""G"",""T""]","""APC""","""NM_000038.6""","""324""","""ENST00000257430.9""",True,True,NA,NA,NA,NA,NA
"""NP_000029.2""",2,"""A""","""T""",chr5:112754894,"[""G"",""A""]","""APC""","""NM_000038.6""","""324""","""ENST00000257430.9""",True,True,NA,NA,NA,NA,NA
"""NP_000029.2""",2,"""A""","""V""",chr5:112754895,"[""C"",""T""]","""APC""","""NM_000038.6""","""324""","""ENST00000257430.9""",True,True,NA,NA,NA,NA,NA
"""NP_000029.2""",3,"""A""","""E""",chr5:112754898,"[""C"",""A""]","""APC""","""NM_000038.6""","""324""","""ENST00000257430.9""",True,True,NA,NA,NA,NA,NA
"""NP_000029.2""",3,"""A""","""G""",chr5:112754898,"[""C"",""G""]","""APC""","""NM_000038.6""","""324""","""ENST00000257430.9""",True,True,NA,NA,NA,NA,NA


In [46]:
popeve_vsmf_missing = popeve_vsmf.aggregate(hl.agg.collect_as_set(popeve_vsmf.NP))

Output()

In [47]:
popeve_vsmf_missing


{'NP_000507.1',
 'NP_003028.1',
 'NP_001035928.3',
 'NP_001129691.1',
 'NP_065174.1',
 'NP_001154889.1',
 'NP_077278.1',
 'NP_001177779.1',
 'NP_002566.1',
 'NP_006435.2',
 'NP_001351637.1',
 'NP_001008391.1',
 'NP_000861.1',
 'NP_004850.1',
 'NP_001166245.1',
 'NP_065896.1',
 'NP_734467.1',
 'NP_001357054.1',
 'NP_006203.2',
 'NP_689911.2',
 'NP_001317230.1',
 'NP_001362513.1',
 'NP_000335.1',
 'NP_005817.1',
 'NP_001365958.1',
 'NP_001374149.1',
 'NP_001374620.1',
 'NP_976313.1',
 'NP_000512.2',
 'NP_277021.2',
 'NP_001340354.1',
 'NP_001324.2',
 'NP_001076437.1',
 'NP_002304.3',
 'NP_002646.2',
 'NP_006810.1',
 'NP_963885.1',
 'NP_001035984.1',
 'NP_002329.2',
 'NP_776189.3',
 'NP_001307799.1',
 'NP_997238.2',
 'NP_001032208.1',
 'NP_001364864.1',
 'NP_060972.3',
 'NP_788276.1',
 'NP_002306.1',
 'NP_060206.2',
 'NP_000152.1',
 'NP_079389.2',
 'NP_001378887.1',
 'NP_008885.1',
 'NP_008998.1',
 'NP_006000.2',
 'NP_597730.2',
 'NP_001361757.1',
 'NP_060691.2',
 'NP_001361.1',
 'NP_0012

In [49]:
l = list(popeve_vsmf_missing)

In [50]:
with open('missing_popeve.txt', 'w') as file:
    for item in l:
        file.write(f"{item}\n")

AttributeError: 'set' object has no attribute 'export'

# Check VSM

In [3]:
vsm_ht = hl.read_table('gs://trisha-tmp/make_linker/linker2_temp_union_final.ht')
vsm_ht = vsm_ht.annotate(
    uniprot_base = hl.if_else(
        vsm_ht.uniprot_em_id.contains('-'), vsm_ht.uniprot_em_id.split('-')[0], hl.missing(hl.tstr)
    )
)

Output()

Output()

In [4]:
vsm_ht.describe()

----------------------------------------
Global fields:
    'grange': array<int32> 
    'vep_help': str 
    'vep_config': str 
    'version': str 
----------------------------------------
Row fields:
    'enst': str 
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'aa_pos': int32 
    'aa_ref': str 
    'aa_alt': str 
    'gene_symbol': str 
    'ensp': str 
    'ensg': str 
    'mane_NM': str 
    'mane_select': bool 
    'canonical': bool 
    'uniprot_em_id': str 
    'transcript_mane_select': bool 
    'uniprot_base': str 
----------------------------------------
Key: ['enst']
----------------------------------------


In [5]:
vsm_ht_k = vsm_ht.key_by('locus', 'alleles')
vsm_ht_k = vsm_ht_k.collect_by_key()
vsm_ht_k.write('gs://trisha-tmp/new_VSM_temp/vsm_ht_k.ht', overwrite=True)

Output()

In [6]:
vsm_ht_k = hl.read_table('gs://trisha-tmp/new_VSM_temp/vsm_ht_k.ht')
vsm_ht_k.show()

Output()

Output()

Output()

,,
locus,alleles,values
locus<GRCh38>,array<str>,"array<struct{enst: str, aa_pos: int32, aa_ref: str, aa_alt: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str}>"
chr1:65568,"[""A"",""C""]","[(""ENST00000641515"",2,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]"
chr1:65568,"[""A"",""G""]","[(""ENST00000641515"",2,""K"",""E"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]"
chr1:65569,"[""A"",""C""]","[(""ENST00000641515"",2,""K"",""T"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]"
chr1:65569,"[""A"",""G""]","[(""ENST00000641515"",2,""K"",""R"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]"
chr1:65569,"[""A"",""T""]","[(""ENST00000641515"",2,""K"",""M"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]"
chr1:65570,"[""G"",""C""]","[(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]"
chr1:65570,"[""G"",""T""]","[(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]"
chr1:65571,"[""A"",""C""]","[(""ENST00000641515"",3,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]"


In [7]:
vsm_ht_k2 = vsm_ht_k.collect_by_key()
vsm_ht_k2.show()

Output()

,,
locus,alleles,values
locus<GRCh38>,array<str>,"array<struct{values: array<struct{enst: str, aa_pos: int32, aa_ref: str, aa_alt: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str}>}>"
chr1:65568,"[""A"",""C""]","[([(""ENST00000641515"",2,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65568,"[""A"",""G""]","[([(""ENST00000641515"",2,""K"",""E"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65569,"[""A"",""C""]","[([(""ENST00000641515"",2,""K"",""T"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65569,"[""A"",""G""]","[([(""ENST00000641515"",2,""K"",""R"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65569,"[""A"",""T""]","[([(""ENST00000641515"",2,""K"",""M"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65570,"[""G"",""C""]","[([(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65570,"[""G"",""T""]","[([(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65571,"[""A"",""C""]","[([(""ENST00000641515"",3,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"


In [8]:
vsm_ht_k2 = vsm_ht_k2.checkpoint('gs://trisha-tmp/new_VSM_temp/vsm_ht_collected.ht', overwrite=True)

Output()

In [10]:
vsm_ht_k2.show()

Output()

,,
locus,alleles,values
locus<GRCh38>,array<str>,"array<struct{values: array<struct{enst: str, aa_pos: int32, aa_ref: str, aa_alt: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str}>}>"
chr1:65568,"[""A"",""C""]","[([(""ENST00000641515"",2,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65568,"[""A"",""G""]","[([(""ENST00000641515"",2,""K"",""E"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65569,"[""A"",""C""]","[([(""ENST00000641515"",2,""K"",""T"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65569,"[""A"",""G""]","[([(""ENST00000641515"",2,""K"",""R"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65569,"[""A"",""T""]","[([(""ENST00000641515"",2,""K"",""M"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65570,"[""G"",""C""]","[([(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65570,"[""G"",""T""]","[([(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"
chr1:65571,"[""A"",""C""]","[([(""ENST00000641515"",3,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)])]"


In [11]:
vsm_ht_k2.describe()

----------------------------------------
Global fields:
    'grange': array<int32> 
    'vep_help': str 
    'vep_config': str 
    'version': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'values': array<struct {
        values: array<struct {
            enst: str, 
            aa_pos: int32, 
            aa_ref: str, 
            aa_alt: str, 
            gene_symbol: str, 
            ensp: str, 
            ensg: str, 
            mane_NM: str, 
            mane_select: bool, 
            canonical: bool, 
            uniprot_em_id: str, 
            transcript_mane_select: bool, 
            uniprot_base: str
        }>
    }> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [ ]:
vsm_ht_k2 = vsm_ht_k.annotate(
    mane_enst = hl.filter(lambda x: x.mane_select == True, vsm_ht_k.values).map(lambda x: x['enst']),
    canon_enst = hl.filter(lambda x: x.canonical == True, vsm_ht_k.values).map(lambda x: x['enst']),
)

In [13]:
vsm_ht_k2.show()

Output()

,,,,
locus,alleles,values,mane_enst,canon_enst
locus<GRCh38>,array<str>,"array<struct{enst: str, aa_pos: int32, aa_ref: str, aa_alt: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str}>",array<str>,array<str>
chr1:65568,"[""A"",""C""]","[(""ENST00000641515"",2,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]"
chr1:65568,"[""A"",""G""]","[(""ENST00000641515"",2,""K"",""E"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]"
chr1:65569,"[""A"",""C""]","[(""ENST00000641515"",2,""K"",""T"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]"
chr1:65569,"[""A"",""G""]","[(""ENST00000641515"",2,""K"",""R"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]"
chr1:65569,"[""A"",""T""]","[(""ENST00000641515"",2,""K"",""M"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]"
chr1:65570,"[""G"",""C""]","[(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]"
chr1:65570,"[""G"",""T""]","[(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]"
chr1:65571,"[""A"",""C""]","[(""ENST00000641515"",3,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]"


In [14]:
vsm_ht_k3 = vsm_ht_k2.annotate(
    mane_len = hl.len(vsm_ht_k2.mane_enst),
    canon_len = hl.len(vsm_ht_k2.canon_enst),
)
vsm_ht_k3.show()

Output()

,,,,,,
locus,alleles,values,mane_enst,canon_enst,mane_len,canon_len
locus<GRCh38>,array<str>,"array<struct{enst: str, aa_pos: int32, aa_ref: str, aa_alt: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str}>",array<str>,array<str>,int32,int32
chr1:65568,"[""A"",""C""]","[(""ENST00000641515"",2,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]",1,1
chr1:65568,"[""A"",""G""]","[(""ENST00000641515"",2,""K"",""E"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]",1,1
chr1:65569,"[""A"",""C""]","[(""ENST00000641515"",2,""K"",""T"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]",1,1
chr1:65569,"[""A"",""G""]","[(""ENST00000641515"",2,""K"",""R"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]",1,1
chr1:65569,"[""A"",""T""]","[(""ENST00000641515"",2,""K"",""M"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]",1,1
chr1:65570,"[""G"",""C""]","[(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]",1,1
chr1:65570,"[""G"",""T""]","[(""ENST00000641515"",2,""K"",""N"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]",1,1
chr1:65571,"[""A"",""C""]","[(""ENST00000641515"",3,""K"",""Q"",""OR4F5"",""ENSP00000493376"",""ENSG00000186092"",""NM_001005484.2"",True,True,""A0A2U3U0J3"",True,NA)]","[""ENST00000641515""]","[""ENST00000641515""]",1,1


In [16]:
vsm_ht_k4 = vsm_ht_k3.filter(vsm_ht_k3.mane_len > 1)

In [17]:
vsm_ht_k4.show()

Output()

,,,,,,
locus,alleles,values,mane_enst,canon_enst,mane_len,canon_len
locus<GRCh38>,array<str>,"array<struct{enst: str, aa_pos: int32, aa_ref: str, aa_alt: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str}>",array<str>,array<str>,int32,int32
chr1:3775377,"[""C"",""A""]","[(""ENST00000444870"",2,""Q"",""K"",""SMIM1"",""ENSP00000457386"",""ENSG00000235169"",NA,False,False,""B2RUZ4"",False,NA),(""ENST00000452264"",2,""Q"",""K"",""SMIM1"",""ENSP00000457674"",""ENSG00000235169"",NA,False,False,NA,False,NA),(""ENST00000561886"",2,""Q"",""K"",""SMIM1"",""ENSP00000456559"",""ENSG00000235169"",NA,False,False,""H3BS66"",False,NA),(""ENST00000642557"",2,""Q"",""K"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA),(""ENST00000642557"",2,""Q"",""K"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA)]","[""ENST00000642557"",""ENST00000642557""]","[""ENST00000642557"",""ENST00000642557""]",2,2
chr1:3775377,"[""C"",""G""]","[(""ENST00000444870"",2,""Q"",""E"",""SMIM1"",""ENSP00000457386"",""ENSG00000235169"",NA,False,False,""B2RUZ4"",False,NA),(""ENST00000452264"",2,""Q"",""E"",""SMIM1"",""ENSP00000457674"",""ENSG00000235169"",NA,False,False,NA,False,NA),(""ENST00000561886"",2,""Q"",""E"",""SMIM1"",""ENSP00000456559"",""ENSG00000235169"",NA,False,False,""H3BS66"",False,NA),(""ENST00000642557"",2,""Q"",""E"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA),(""ENST00000642557"",2,""Q"",""E"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA)]","[""ENST00000642557"",""ENST00000642557""]","[""ENST00000642557"",""ENST00000642557""]",2,2
chr1:3775378,"[""A"",""C""]","[(""ENST00000444870"",2,""Q"",""P"",""SMIM1"",""ENSP00000457386"",""ENSG00000235169"",NA,False,False,""B2RUZ4"",False,NA),(""ENST00000452264"",2,""Q"",""P"",""SMIM1"",""ENSP00000457674"",""ENSG00000235169"",NA,False,False,NA,False,NA),(""ENST00000561886"",2,""Q"",""P"",""SMIM1"",""ENSP00000456559"",""ENSG00000235169"",NA,False,False,""H3BS66"",False,NA),(""ENST00000642557"",2,""Q"",""P"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA),(""ENST00000642557"",2,""Q"",""P"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA)]","[""ENST00000642557"",""ENST00000642557""]","[""ENST00000642557"",""ENST00000642557""]",2,2
chr1:3775378,"[""A"",""G""]","[(""ENST00000444870"",2,""Q"",""R"",""SMIM1"",""ENSP00000457386"",""ENSG00000235169"",NA,False,False,""B2RUZ4"",False,NA),(""ENST00000452264"",2,""Q"",""R"",""SMIM1"",""ENSP00000457674"",""ENSG00000235169"",NA,False,False,NA,False,NA),(""ENST00000561886"",2,""Q"",""R"",""SMIM1"",""ENSP00000456559"",""ENSG00000235169"",NA,False,False,""H3BS66"",False,NA),(""ENST00000642557"",2,""Q"",""R"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA),(""ENST00000642557"",2,""Q"",""R"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA)]","[""ENST00000642557"",""ENST00000642557""]","[""ENST00000642557"",""ENST00000642557""]",2,2
chr1:3775378,"[""A"",""T""]","[(""ENST00000444870"",2,""Q"",""L"",""SMIM1"",""ENSP00000457386"",""ENSG00000235169"",NA,False,False,""B2RUZ4"",False,NA),(""ENST00000452264"",2,""Q"",""L"",""SMIM1"",""ENSP00000457674"",""ENSG00000235169"",NA,False,False,NA,False,NA),(""ENST00000561886"",2,""Q"",""L"",""SMIM1"",""ENSP00000456559"",""ENSG00000235169"",NA,False,False,""H3BS66"",False,NA),(""ENST00000642557"",2,""Q"",""L"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA),(""ENST00000642557"",2,""Q"",""L"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA)]

In [20]:
vsm_ht_k5 = vsm_ht_k4.annotate(
    mane_enst_set = hl.array(hl.set(vsm_ht_k4.mane_enst)),
    canon_enst_set = hl.array(hl.set(vsm_ht_k4.canon_enst)),
)


In [18]:
vsm_ht_k4_sm = vsm_ht_k4.head(1)

In [19]:
vsm_ht_k4_sm.explode(vsm_ht_k4_sm.values).show()

Output()

+---------------+------------+-------------------+---------------+
| locus         | alleles    | values.enst       | values.aa_pos |
+---------------+------------+-------------------+---------------+
| locus<GRCh38> | array<str> | str               |         int32 |
+---------------+------------+-------------------+---------------+
| chr1:3775377  | ["C","A"]  | "ENST00000444870" |             2 |
| chr1:3775377  | ["C","A"]  | "ENST00000452264" |             2 |
| chr1:3775377  | ["C","A"]  | "ENST00000561886" |             2 |
| chr1:3775377  | ["C","A"]  | "ENST00000642557" |             2 |
| chr1:3775377  | ["C","A"]  | "ENST00000642557" |             2 |
+---------------+------------+-------------------+---------------+

+---------------+---------------+--------------------+-------------------+
| values.aa_ref | values.aa_alt | values.gene_symbol | values.ensp       |
+---------------+---------------+--------------------+-------------------+
| str           | str           | str                | str               |
+---------------+---------------+--------------------+-------------------+
| "Q"           | "K"           | "SMIM1"            | "ENSP00000457386" |
| "Q"           | "K"           | "SMIM1"            | "ENSP00000457674" |
| "Q"           | "K"           | "SMIM1"            | "ENSP00000456559" |
| "Q"           | "K"           | "SMIM1"            | "ENSP00000496314" |
| "Q"           | "K"           | "SMIM1"            | "ENSP00000496314" |
+---------------+---------------+--------------------+-------------------+

+-------------------+------------------+--------------------+------------------+
| values.ensg       | values.mane_NM   | values.mane_select | values.canonical |
+-------------------+------------------+--------------------+------------------+
| str               | str              |               bool |             bool |
+-------------------+------------------+--------------------+------------------+
| "ENSG00000235169" | NA               |              False |            False |
| "ENSG00000235169" | NA               |              False |            False |
| "ENSG00000235169" | NA               |              False |            False |
| "ENSG00000235169" | "NM_001288583.2" |               True |             True |
| "ENSG00000235169" | "NM_001288583.2" |               True |             True |
+-------------------+------------------+--------------------+------------------+

+----------------------+-------------------------------+---------------------+
| values.uniprot_em_id | values.transcript_mane_select | values.uniprot_base |
+----------------------+-------------------------------+---------------------+
| str                  |                          bool | str                 |
+----------------------+-------------------------------+---------------------+
| "B2RUZ4"             |                         False | NA                  |
| NA                   |                         False | NA                  |
| "H3BS66"             |                         False | NA                  |
| "B2RUZ4"             |                          True | NA                  |
| "B2RUZ4"             |                          True | NA                  |
+----------------------+-------------------------------+---------------------+

+---------------------------------------+
| mane_enst                             |
+---------------------------------------+
| array<str>                            |
+---------------------------------------+
| ["ENST00000642557","ENST00000642557"] |
| ["ENST00000642557","ENST00000642557"] |
| ["ENST00000642557","ENST00000642557"] |
| ["ENST00000642557","ENST00000642557"] |
| ["ENST00000642557","ENST00000642557"] |
+---------------------------------------+

+---------------------------------------+----------+-----------+
| canon_enst                            | mane_len | canon_len |
+---------------------------------------+----------+-----------+
| array<str>    

In [21]:
vsm_ht_k5.show()

Output()

,,,,,,,,
locus,alleles,values,mane_enst,canon_enst,mane_len,canon_len,mane_enst_set,canon_enst_set
locus<GRCh38>,array<str>,"array<struct{enst: str, aa_pos: int32, aa_ref: str, aa_alt: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str}>",array<str>,array<str>,int32,int32,array<str>,array<str>
chr1:3775377,"[""C"",""A""]","[(""ENST00000444870"",2,""Q"",""K"",""SMIM1"",""ENSP00000457386"",""ENSG00000235169"",NA,False,False,""B2RUZ4"",False,NA),(""ENST00000452264"",2,""Q"",""K"",""SMIM1"",""ENSP00000457674"",""ENSG00000235169"",NA,False,False,NA,False,NA),(""ENST00000561886"",2,""Q"",""K"",""SMIM1"",""ENSP00000456559"",""ENSG00000235169"",NA,False,False,""H3BS66"",False,NA),(""ENST00000642557"",2,""Q"",""K"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA),(""ENST00000642557"",2,""Q"",""K"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA)]","[""ENST00000642557"",""ENST00000642557""]","[""ENST00000642557"",""ENST00000642557""]",2,2,"[""ENST00000642557""]","[""ENST00000642557""]"
chr1:3775377,"[""C"",""G""]","[(""ENST00000444870"",2,""Q"",""E"",""SMIM1"",""ENSP00000457386"",""ENSG00000235169"",NA,False,False,""B2RUZ4"",False,NA),(""ENST00000452264"",2,""Q"",""E"",""SMIM1"",""ENSP00000457674"",""ENSG00000235169"",NA,False,False,NA,False,NA),(""ENST00000561886"",2,""Q"",""E"",""SMIM1"",""ENSP00000456559"",""ENSG00000235169"",NA,False,False,""H3BS66"",False,NA),(""ENST00000642557"",2,""Q"",""E"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA),(""ENST00000642557"",2,""Q"",""E"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA)]","[""ENST00000642557"",""ENST00000642557""]","[""ENST00000642557"",""ENST00000642557""]",2,2,"[""ENST00000642557""]","[""ENST00000642557""]"
chr1:3775378,"[""A"",""C""]","[(""ENST00000444870"",2,""Q"",""P"",""SMIM1"",""ENSP00000457386"",""ENSG00000235169"",NA,False,False,""B2RUZ4"",False,NA),(""ENST00000452264"",2,""Q"",""P"",""SMIM1"",""ENSP00000457674"",""ENSG00000235169"",NA,False,False,NA,False,NA),(""ENST00000561886"",2,""Q"",""P"",""SMIM1"",""ENSP00000456559"",""ENSG00000235169"",NA,False,False,""H3BS66"",False,NA),(""ENST00000642557"",2,""Q"",""P"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA),(""ENST00000642557"",2,""Q"",""P"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA)]","[""ENST00000642557"",""ENST00000642557""]","[""ENST00000642557"",""ENST00000642557""]",2,2,"[""ENST00000642557""]","[""ENST00000642557""]"
chr1:3775378,"[""A"",""G""]","[(""ENST00000444870"",2,""Q"",""R"",""SMIM1"",""ENSP00000457386"",""ENSG00000235169"",NA,False,False,""B2RUZ4"",False,NA),(""ENST00000452264"",2,""Q"",""R"",""SMIM1"",""ENSP00000457674"",""ENSG00000235169"",NA,False,False,NA,False,NA),(""ENST00000561886"",2,""Q"",""R"",""SMIM1"",""ENSP00000456559"",""ENSG00000235169"",NA,False,False,""H3BS66"",False,NA),(""ENST00000642557"",2,""Q"",""R"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA),(""ENST00000642557"",2,""Q"",""R"",""SMIM1"",""ENSP00000496314"",""ENSG00000235169"",""NM_001288583.2"",True,True,""B2RUZ4"",True,NA)]","[""ENST00000642557"",""ENST00000642557""]","[""ENST00000642557"",""ENST00000642557""]",2,2,"[""ENST00000642557""]","[""ENST00000642557""]"
chr1:3775378,"[""A"",""T""]","[(""ENST00000444870"",2,""Q"",""L"",""SMIM1"",""ENSP00000457386"",""ENSG00000235169"",NA,False,False,""B2RUZ4"",False,NA),(""ENST00000452264"",2,""Q"",""L"",""SMIM1"",""ENSP00000457674"",""ENSG00000235169"",NA,False,False,NA,False,NA),(""ENST00000561886"",2,""Q"",""L"",""SMIM1"",""ENSP00000456559"",""ENSG00000235169"",NA,False,False,""H3BS66"",False,NA),(""ENST00000642557"",2,

In [24]:
vsm_ht_k6 = vsm_ht_k5.filter(hl.len(vsm_ht_k5.mane_enst_set) > 1)

In [25]:
vsm_ht_k6.show()

Output()

,,,,,,,,
locus,alleles,values,mane_enst,canon_enst,mane_len,canon_len,mane_enst_set,canon_enst_set
locus<GRCh38>,array<str>,"array<struct{enst: str, aa_pos: int32, aa_ref: str, aa_alt: str, gene_symbol: str, ensp: str, ensg: str, mane_NM: str, mane_select: bool, canonical: bool, uniprot_em_id: str, transcript_mane_select: bool, uniprot_base: str}>",array<str>,array<str>,int32,int32,array<str>,array<str>
chr1:111503005,"[""C"",""A""]","[(""ENST00000241356"",117,""R"",""I"",""ADORA3"",""ENSP00000241356"",""ENSG00000282608"",""NM_000677.4"",True,True,""P0DMS8-1"",True,""P0DMS8""),(""ENST00000369716"",117,""R"",""I"",""TMIGD3"",""ENSP00000358730"",""ENSG00000121933"",""NM_020683.7"",True,True,""P0DMS9-2"",True,""P0DMS9""),(""ENST00000632535"",117,""S"",""I"",""ADORA3"",""ENSP00000488073"",""ENSG00000282608"",NA,False,False,""A0A0J9YWR0"",False,NA)]","[""ENST00000241356"",""ENST00000369716""]","[""ENST00000241356"",""ENST00000369716""]",2,2,"[""ENST00000241356"",""ENST00000369716""]","[""ENST00000241356"",""ENST00000369716""]"
chr1:111503005,"[""C"",""G""]","[(""ENST00000241356"",117,""R"",""T"",""ADORA3"",""ENSP00000241356"",""ENSG00000282608"",""NM_000677.4"",True,True,""P0DMS8-1"",True,""P0DMS8""),(""ENST00000369716"",117,""R"",""T"",""TMIGD3"",""ENSP00000358730"",""ENSG00000121933"",""NM_020683.7"",True,True,""P0DMS9-2"",True,""P0DMS9""),(""ENST00000632535"",117,""S"",""T"",""ADORA3"",""ENSP00000488073"",""ENSG00000282608"",NA,False,False,""A0A0J9YWR0"",False,NA)]","[""ENST00000241356"",""ENST00000369716""]","[""ENST00000241356"",""ENST00000369716""]",2,2,"[""ENST00000241356"",""ENST00000369716""]","[""ENST00000241356"",""ENST00000369716""]"
chr1:111503005,"[""C"",""T""]","[(""ENST00000241356"",117,""R"",""K"",""ADORA3"",""ENSP00000241356"",""ENSG00000282608"",""NM_000677.4"",True,True,""P0DMS8-1"",True,""P0DMS8""),(""ENST00000369716"",117,""R"",""K"",""TMIGD3"",""ENSP00000358730"",""ENSG00000121933"",""NM_020683.7"",True,True,""P0DMS9-2"",True,""P0DMS9""),(""ENST00000632535"",117,""S"",""N"",""ADORA3"",""ENSP00000488073"",""ENSG00000282608"",NA,False,False,""A0A0J9YWR0"",False,NA)]","[""ENST00000241356"",""ENST00000369716""]","[""ENST00000241356"",""ENST00000369716""]",2,2,"[""ENST00000241356"",""ENST00000369716""]","[""ENST00000241356"",""ENST00000369716""]"
chr1:111503006,"[""T"",""C""]","[(""ENST00000241356"",117,""R"",""G"",""ADORA3"",""ENSP00000241356"",""ENSG00000282608"",""NM_000677.4"",True,True,""P0DMS8-1"",True,""P0DMS8""),(""ENST00000369716"",117,""R"",""G"",""TMIGD3"",""ENSP00000358730"",""ENSG00000121933"",""NM_020683.7"",True,True,""P0DMS9-2"",True,""P0DMS9""),(""ENST00000632535"",117,""S"",""G"",""ADORA3"",""ENSP00000488073"",""ENSG00000282608"",NA,False,False,""A0A0J9YWR0"",False,NA)]","[""ENST00000241356"",""ENST00000369716""]","[""ENST00000241356"",""ENST00000369716""]",2,2,"[""ENST00000241356"",""ENST00000369716""]","[""ENST00000241356"",""ENST00000369716""]"
chr1:111503008,"[""A"",""C""]","[(""ENST00000241356"",116,""V"",""G"",""ADORA3"",""ENSP00000241356"",""ENSG00000282608"",""NM_000677.4"",True,True,""P0DMS8-1"",True,""P0DMS8""),(""ENST00000369716"",116,""V"",""G"",""TMIGD3"",""ENSP00000358730"",""ENSG00000121933"",""NM_020683.7"",True,True,""P0DMS9-2"",True,""P0DMS9""),(""ENST00000632535"",116,""V"",""G"",""ADORA3"",""ENSP00000488073"",""ENSG00000282608"",NA,False,False,""A0A0J9YWR0"",False,NA)]","[""ENST00000241356"",""ENST00000369716""]","[""ENST00000241356"",""ENST00000369716""]",2,2,"[""ENST00000241356"",""ENST00000369716""]","[""ENST00000241356"",""ENST00000369716""]"
chr1:111503008,"[""A"",""G""]","[(""ENST00000241356"",116,""V"",""A"",""ADORA3"",""ENSP00000241356"",""ENSG00000282608"",""NM_000677.4"",True,True,""P0DMS8-1"",True,""P0DMS8""),(""ENST00000369716"",116,""V"",""A"",""TMIGD3"",""ENSP00000358730"",""ENSG00000121933"",""NM_020683.7"",True,True,""P0DMS9-2"",True,""P0DMS9""),(""ENST00000632535"",116,""V"",""A"",""ADORA3"",""ENSP00000488073"",""ENSG00000282608"",NA,False,False,""A0A0J9YWR0""

In [27]:
vsm_ht_k6.count()

Output()

9220

In [28]:
vsm_ht_k7 = vsm_ht_k6.annotate(
    first_mane_enst = vsm_ht_k6.mane_enst[0],
)
num_transcripts = vsm_ht_k7.aggregate(hl.agg.collect_as_set(vsm_ht_k7.first_mane_enst))
num_transcripts

Output()

{'ENST00000229266',
 'ENST00000241356',
 'ENST00000252085',
 'ENST00000252136',
 'ENST00000253807',
 'ENST00000274766',
 'ENST00000277165',
 'ENST00000286604',
 'ENST00000300036',
 'ENST00000300811',
 'ENST00000302450',
 'ENST00000305139',
 'ENST00000326441',
 'ENST00000354588',
 'ENST00000367615',
 'ENST00000370597',
 'ENST00000378719',
 'ENST00000530210'}

In [29]:
len(num_transcripts)

18

In [30]:
vsm_ht_k7_sm = vsm_ht_k7.head(1)

In [31]:
vsm_ht_k7_sm.explode(vsm_ht_k7_sm.values).show()

Output()

+----------------+------------+-------------------+---------------+
| locus          | alleles    | values.enst       | values.aa_pos |
+----------------+------------+-------------------+---------------+
| locus<GRCh38>  | array<str> | str               |         int32 |
+----------------+------------+-------------------+---------------+
| chr1:111503005 | ["C","A"]  | "ENST00000241356" |           117 |
| chr1:111503005 | ["C","A"]  | "ENST00000369716" |           117 |
| chr1:111503005 | ["C","A"]  | "ENST00000632535" |           117 |
+----------------+------------+-------------------+---------------+

+---------------+---------------+--------------------+-------------------+
| values.aa_ref | values.aa_alt | values.gene_symbol | values.ensp       |
+---------------+---------------+--------------------+-------------------+
| str           | str           | str                | str               |
+---------------+---------------+--------------------+-------------------+
| "R"           | "I"           | "ADORA3"           | "ENSP00000241356" |
| "R"           | "I"           | "TMIGD3"           | "ENSP00000358730" |
| "S"           | "I"           | "ADORA3"           | "ENSP00000488073" |
+---------------+---------------+--------------------+-------------------+

+-------------------+----------------+--------------------+------------------+
| values.ensg       | values.mane_NM | values.mane_select | values.canonical |
+-------------------+----------------+--------------------+------------------+
| str               | str            |               bool |             bool |
+-------------------+----------------+--------------------+------------------+
| "ENSG00000282608" | "NM_000677.4"  |               True |             True |
| "ENSG00000121933" | "NM_020683.7"  |               True |             True |
| "ENSG00000282608" | NA             |              False |            False |
+-------------------+----------------+--------------------+------------------+

+----------------------+-------------------------------+---------------------+
| values.uniprot_em_id | values.transcript_mane_select | values.uniprot_base |
+----------------------+-------------------------------+---------------------+
| str                  |                          bool | str                 |
+----------------------+-------------------------------+---------------------+
| "P0DMS8-1"           |                          True | "P0DMS8"            |
| "P0DMS9-2"           |                          True | "P0DMS9"            |
| "A0A0J9YWR0"         |                         False | NA                  |
+----------------------+-------------------------------+---------------------+

+---------------------------------------+
| mane_enst                             |
+---------------------------------------+
| array<str>                            |
+---------------------------------------+
| ["ENST00000241356","ENST00000369716"] |
| ["ENST00000241356","ENST00000369716"] |
| ["ENST00000241356","ENST00000369716"] |
+---------------------------------------+

+---------------------------------------+----------+-----------+
| canon_enst                            | mane_len | canon_len |
+---------------------------------------+----------+-----------+
| array<str>                            |    int32 |     int32 |
+---------------------------------------+----------+-----------+
| ["ENST00000241356","ENST00000369716"] |        2 |         2 |
| ["ENST00000241356","ENST00000369716"] |        2 |         2 |
| ["ENST00000241356","ENST00000369716"] |        2 |         2 |
+---------------------------------------+----------+-----------+

+---------------------------------------+
| mane_enst_set                         |
+---------------------------------------+
| array<str>                            |
+---------------------------------------+
| ["ENST00000241356","ENST00000369716"] |
| ["ENST00000241356","ENST00000369716"] |
| ["ENST00000241356","ENST00